# Data Understanding & Data Preparation

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
# Show all rows
pd.set_option('display.max_rows', None)

# Show all columns
pd.set_option('display.max_columns', None)

# Show full column contents (no truncation)
pd.set_option('display.max_colwidth', None)

# Don’t wrap long output lines
pd.set_option('display.expand_frame_repr', False)

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import TwoSlopeNorm

import seaborn as sns
import re
import altair as alt
import dtale
import plotly.express as px

import project_fuctions as functions

import ast
from langdetect import detect, DetectorFactory

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


DetectorFactory.seed = 42

alt.data_transformers.disable_max_rows()


## Reading data

In [ ]:
artists_path = 'data\\artists.csv'
tracks_path = 'data\\tracks.csv'

This code automatically detects the correct separator for two dataset files (tracks and artists) by checking which character — comma, semicolon, or tab — appears most in the first line. It then loads each file into a pandas DataFrame using the detected separator, prints their shapes, and displays the first few rows.

 The tracks dataset has 11,166 rows and 45 columns, while the artists dataset has 104 rows and 14 columns.

In [ ]:
# Funzione helper per capire il separatore corretto
def detect_separator(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        sample = f.readline()
    # Conta quanti separatori compaiono
    seps = {',': sample.count(','), ';': sample.count(';'), '\t': sample.count('\t')}
    best_sep = max(seps, key=seps.get)
    print(f"Detected separator for {filepath}: '{best_sep}'")
    return best_sep

# Rileva automaticamente il separatore
sep_tracks = detect_separator(tracks_path)
sep_artists = detect_separator(artists_path)

print('------------------------------------')

# Carica i dataset in base al separatore rilevato
tracks = pd.read_csv(tracks_path, sep=sep_tracks, encoding='utf-8', engine='python')
artists = pd.read_csv(artists_path, sep=sep_artists, encoding='utf-8', engine='python')

df = tracks.merge(
    artists,
    left_on="id_artist",
    right_on="id_author",
    how="left",
    suffixes=("", "_artist")
)

# Mostra alcune info per verifica
print(f"Tracks shape: {tracks.shape}")
print(f"Artists shape: {artists.shape}")
print("Shape df (merged):", df.shape)
print('------------------------------------')

print('TRACKS')
display(tracks.head(3))

print('------------------------------------')
print('ARTISTS')
display(artists.head(3))

print('------------------------------------')
print('MERGERD')
display(df.head(3))

## DATA UNDERSTANDING

Objectives:
- Describe the `all_tracks.csv` and `artist.csv` datasets.
- Document the meaning of the variables (data dictionary).
- Analyze data quality: types, missing values, duplicates.
- Explore distributions and relationships between relevant features.

All of this will be included in the "Data Understanding" section of the report.

In [ ]:
print("=== INFO all_tracks ===")
tracks.info()

In [ ]:
print("\n=== INFO artist ===")
artists.info()

In [ ]:
print("\n=== INFO df (merged) ===")
df.info()

### Data Distribution
The following table and histogram show the numerical data distribution in the dataset:

- **Most features** (`n_sentences`, `n_tokens`, `tokens_per_sent`, `char_per_tok`, `lexical_density`, `avg_token_per_clause`, `centroid`, `rolloff`, `rms`, `zcr`, `flatness`, `flux`, `spectral_complexity`, `pitch`, `loudness`) show **bell-shaped or near-normal distributions**.

- **Highly skewed features** (`stats_pageviews`, `bpm`, `tokens_per_sent`, `duration_ms`, `popularity`) have a **long right tail**, indicating a few extreme values or outliers (common in popularity or count-based features).

- **Temporal features** (`year`, `month`, `day`) display **non-uniform distributions**; e.g., `year` is concentrated around recent decades, showing most songs are modern.

- **Geographical features** (`latitude`, `longitude`) have **peaks corresponding to specific locations**, likely representing where artists or tracks are clustered.

The data distribution and the statistics presented above reveal some anomalies and irregularities in the dataset. These issues will be examined and addressed in the following section.

In [ ]:

# Select numeric columns
num_cols = df.select_dtypes(include=['float64', 'int64']).columns

# --- Summary statistics table ---
display(df[num_cols].describe().T.style.background_gradient(cmap='RdPu'))

# --- Histograms for each numeric column ---
n_cols = 4
n_rows = -(-len(num_cols) // n_cols)  # ceil division
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    sns.histplot(df[col].dropna(), bins=30, kde=True, color="#d36ba8", ax=axes[i])
    axes[i].set_title(col, fontsize=18, color="#b30059")   # larger title font
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")
    axes[i].tick_params(axis='both', labelsize=12)          # larger tick labels

# Remove unused axes
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("Distribution of Numerical Features", fontsize=24, color="#000000", y=1.02)  # larger main title
plt.tight_layout()
plt.show()


In [ ]:
# Calculate basic statistics (mean and median)
statistics = df[num_cols].describe().T

# Calculate skewness
skews = df[num_cols].skew()

skew_analysis = pd.DataFrame({
    'mean': statistics['mean'],
    'median': statistics['50%'],
    'skewness_value': skews
})

# Define a function to classify skewness
# These are standard thresholds used in statistics:
# > +0.5 = Positive Skew (Right-tailed)
# < -0.5 = Negative Skew (Left-tailed)
# Between -0.5 and +0.5 = Substantially Symmetric
def classify_skew(skew_value):
    if skew_value > 0.5:
        return "Positive (Right Skew)"
    elif skew_value < -0.5:
        return "Negative (Left Skew)"
    else:
        return "Symmetric"

skew_analysis['skew_type'] = skew_analysis['skewness_value'].apply(classify_skew)

print("Skewness Analysis of Numerical Features")

display(skew_analysis.sort_values(by='skewness_value', ascending=False))

## DATA QUALITY CHECK

### Discovering Missing Value

This code analyzes missing values in the DataFrame by counting how many entries are NaN for each column and calculating the corresponding percentage. It creates a summary table showing only columns with missing data, sorted by the highest percentage.

In [ ]:
# Calcolo missing values e percentuali
missing_count = df.isna().sum()
missing_percent = (missing_count / len(df)) * 100

missing_df = (
    pd.DataFrame({'missing_count': missing_count, 'missing_percent': missing_percent})
    .sort_values('missing_percent', ascending=False)
    .query('missing_percent > 0')
)

# Mostra tabella riepilogativa (gradiente rosso-magenta)
display(
    missing_df
    .style.background_gradient(subset=['missing_percent'], cmap='RdPu')  
    .format({'missing_percent': '{:.2f}%'})
)


The following heatmap visualizes missing values in the dataset, with each row representing a record and each column a feature. Colored cells indicate missing entries, providing a clear overview of where data is incomplete.


In [ ]:
# Trasformiamo il dataframe in formato long (necessario per Altair)
na_df = df.isna().reset_index().melt(id_vars='index')
na_df.columns = ['row', 'column', 'na']

chart = alt.Chart(na_df).mark_rect().encode(
    x=alt.X('column:N', title=None),
    y=alt.Y('row:O', title=None, axis=None),  # rimuove etichette righe come seaborn
    color=alt.Color('na:N',
        scale=alt.Scale(range=["#8e44ad", "#f3e5f5"]),  # viola scuro → viola chiaro
        legend=alt.Legend(title="Missing")
    )
).properties(
    width=600,
    height=300,
    title="Missing Values Matrix (Altair)"
)

chart


The following bar plot shows the percentage of missing values per feature, with the top 20 features that have the most missing data.

In [ ]:
# Prendiamo solo le prime 20 feature (come nel seaborn)
missing_top20 = missing_df.reset_index().rename(columns={'index':'feature'})

# Grafico: barre orizzontali viola
bars = (
    alt.Chart(missing_top20)
    .mark_bar()
    .encode(
        x=alt.X("missing_percent:Q", title="Missing values (%)"),
        y=alt.Y("feature:N", sort='-x', title="Feature name"),
        color=alt.Color(
            "feature:N",
            scale=alt.Scale(range=["#f3e5f5", "#e1bee7", "#ce93d8", "#ba68c8", "#9c27b0"][::-1]),   # palette viola
            legend=None
        ),
        tooltip=[
            alt.Tooltip("feature", title="Feature"),
            alt.Tooltip("missing_percent", title="Missing (%)", format=".2f")
        ]
    )
)

# Testo con la percentuale sulla barra
labels = (
    alt.Chart(missing_top20)
    .mark_text(align="left", baseline="middle", dx=5, color="#4A0038")
    .encode(
        x="missing_percent:Q",
        y="feature:N",
        text=alt.Text("missing_percent:Q", format=".1f")
    )
)

chart = (bars + labels).properties(
    title="Percentage of Missing Values by Feature",
    width=550,
    height=450
)

chart

#### Missing Values Propagation After Merge

In [ ]:
artists_missing = artists.isna().mean().sort_values(ascending=False) * 100
print(artists_missing)

The visualization highlights that missing values in attributes such as active_start, region, and birth_place have increased after merging due to the replication of incomplete artist metadata across multiple tracks.
This confirms that the merge process did not introduce new nulls, but propagated pre-existing ones.

In [ ]:
# Colonne provenienti dal dataset artists 
artist_cols = list(artists.columns)

# Conta i NaN prima e dopo il merge
missing_before = artists[artist_cols].isna().sum()
missing_after = df[artist_cols].isna().sum()

# Differenza assoluta e percentuale
missing_diff = missing_after - missing_before
increase_percent = (missing_diff / missing_before.replace(0, pd.NA)) * 100

# Tabella riepilogativa
missing_summary = (
    pd.DataFrame({
        "missing_before": missing_before,
        "missing_after": missing_after,
        "difference": missing_diff,
        "increase_%": increase_percent
    })
    .sort_values("difference", ascending=False)
)

# --- PRENDI TUTTE LE COLONNE, non solo difference > 0 ---
plot_df = missing_summary.copy()

plot_df = plot_df.reset_index().rename(columns={"index": "feature"})

chart = (
    alt.Chart(plot_df)
    .mark_bar()
    .encode(
        x=alt.X("difference:Q", title="Variation in number of missing values (after - before)"),
        y=alt.Y("feature:N", sort='-x', title="Feature"),
        color=alt.Color(
            "feature:N",
            scale=alt.Scale(range=["#f3e5f5", "#e1bee7", "#ce93d8", "#ba68c8", "#9c27b0"][::-1])
        ),
        tooltip=["feature", "difference", "missing_before", "missing_after", "increase_%"]
    )
    .properties(
        width=600,
        height=300,
        title="Missing Values Variation After Merge (All Features)"
    )
)

chart


After merging the datasets, several artist-related features experienced a significant increase in missing values. This indicates that many records in the merged dataset do not have matching information in the original artists' dataset. Additionally, some features (e.g., active_end, featured_artists, stats_pageviews) show very high percentages of missing data, making them unreliable for analysis. Columns with moderate missing rates (around 20–30%) may still be usable after applying appropriate imputation techniques, while highly incomplete features should be removed or excluded from modeling to avoid introducing noise.

### Duplicates

#### Artists

The following code checks the artists dataset for duplicates in two ways: first, it looks for identical full rows to detect any completely repeated entries; then, it checks for duplicates specifically based on the artist ID and artist name columns.
<B> After performing both checks, it confirms that there are no duplicate artists in the dataset </B>.

In [ ]:
# Check for duplicated artists rows
duplicates_artists = artists[artists.duplicated()]

print(f"Number of duplicated Artists rows: {duplicates_artists.shape[0]}")
display(duplicates_artists.head(5))

In [ ]:
# Check for duplicated artists based on artist id
duplicates_artists_id = artists[artists.duplicated(subset='id_author')]
print(f"Number of duplicated artist based on ID: {duplicates_artists_id.shape[0]}")
display(duplicates_artists_id.head(5))

In [ ]:
# Check for duplicated artists based on artist name	
duplicates_artists_name = artists[artists.duplicated(subset='name')]
print(f"Number of duplicated artist based on Name: {duplicates_artists_name.shape[0]}")
display(duplicates_artists_name.head(5))

#### Tracks
Duplicates rows check has been also performed here.
No duplicated rows were detected, indicating that all track entries are unique.

In [ ]:
# Check for duplicated tracks rows
duplicates_tracks = tracks[tracks.duplicated()]

print(f"Number of duplicated rows: {duplicates_tracks.shape[0]}")
display(duplicates_tracks.head(5))

##### Duplicated Tracks based on ID

This code checks the tracks dataset for duplicates based specifically on the track ID column. It identifies all rows where the same ID appears more than once, counts them, and displays them.
 It first identifies all rows where the same ID appears more than once, counts how many duplicated tracks exist, and displays them. Then, it counts how many times each track ID occurs in the dataset. 

<B> The result shows that there are 73 duplicated rows based on track IDs. 
Precisely we have 71  distinct IDs that have duplicates. </B>

<B>one track ID is repeated four times, while the others are each repeated twice </B>

In [ ]:
# Check for duplicated tracks based on track id
duplicates_tracks_id = tracks[tracks.duplicated(subset='id')]
print(f"Number of duplicated Tracks rows based on ID: {duplicates_tracks_id.shape[0]}")
display(duplicates_tracks_id)


# Count how many times each id_track appears
id_counts = tracks['id'].value_counts()
duplicate_id_counts = id_counts[id_counts > 1]

print('Number of distinct IDs that have duplicates')
print(duplicate_id_counts.size)
print("Number of tracks for each id:")
print(duplicate_id_counts)


The following code lists every full_title associated with each duplicated track ID. The results show 71 duplicated IDs in total. Most of these IDs are linked to two different songs, except for one ID that is associated with four songs (two pairs sharing the same title).

In [ ]:
# Find all duplicated track IDs
duplicate_ids = tracks[tracks.duplicated(subset='id', keep=False)]

# Group by 'id' and list all titles
titles_per_id = duplicate_ids.groupby('id')['full_title'].apply(list)

# Display each ID with all titles and the count of unique titles
for track_id, titles in titles_per_id.items():
    unique_count = len(set(titles))  # number of unique titles
    print(f"Track ID: {track_id} Number(of total songs: {len(titles)})(Unique titles: {unique_count})")
    for title in titles:
        print(f"  - {title}")
    print('----------------------------------------------------------')


##### Duplicated Tracks based on Title
The following code identifies tracks that share the same full_title, meaning duplicate song titles. We found four duplicated tracks, corresponding to two pairs of songs with identical titles.

In [ ]:
# Find duplicated full_title
duplicate_titles = tracks[tracks.duplicated(subset='full_title', keep=False)]

# Sort by full_title to see them together
duplicate_titles = duplicate_titles.sort_values('full_title')

print(f"Tracks with duplicate track based on full_title: {duplicate_titles.shape[0]}")
display(duplicate_titles)


##### Duplicates Tracks base on Lyrics
This code checks the tracks dataset for duplicates based specifically on the lyrics content. As identical lyrics create identical values for all derived numeric features (n_tokens, lexical_density, swear_IT, etc.).

<B> The result shows that there are 12 duplicate rows. </B>

<B> A deeper inspection of the 23 total rows involved reveals two distinct issues: </B>

10 unique sets of lyrics (totaling 20 rows) are logical duplicates. As observed, these include both inter-album duplicates (e.g., 'album' vs 'single' versions) and intra-album variations (e.g., HOT and HOT (Opera) on the same album, or Risatatà RMX and its original).

3 rows are identified as duplicates because their lyrics value is NaN.

<B> Therefore, the 12 duplicates are composed of 10 text-based logical duplicates and 2 NaN duplicates (from the group of 3). Removing these is essential to prevent skewing all derived textual features and the clustering analysis. </B>

In [ ]:
print("Checking for duplicates based on 'lyrics' content...")

# Count the number of rows that are duplicates based on lyrics
lyrics_duplicates_count = df.duplicated(subset=['lyrics']).sum()
print(f"Number of duplicate rows (same 'lyrics'): {lyrics_duplicates_count}")

# Get all rows that have a duplicated lyric to inspect them
duplicated_lyrics_rows = df[df.duplicated(subset=['lyrics'], keep=False)]

if not duplicated_lyrics_rows.empty:
    print(f"\nFound {duplicated_lyrics_rows.shape[0]} total rows involved in lyric duplication.")
    print("Showing some examples, sorted by lyrics to group them:")

    # Display relevant columns to confirm they are logical duplicates
    relevant_cols = [
        'full_title', 'primary_artist', 'album_name', 'album_type',
        'lyrics', 'n_tokens', 'lexical_density', 'swear_IT'
    ]

    # Filter for columns that actually exist in the dataframe
    display_cols = [col for col in relevant_cols if col in df.columns]

    display(
        duplicated_lyrics_rows.sort_values('lyrics')[display_cols].head(23)
    )
else:
    print("OK: No duplicate lyrics found.")

#### Redundant Features

#### Album and Album name

The goal is to understand if the two coloums have the same information. In order to do it we analize:
1. quality and consistency of `album` and `album_name`
2. the relationship with `id_album`

In [ ]:
if "album" in df.columns and "album_name" in df.columns:
    same_raw = (df["album"] == df["album_name"])
    print(f"Match (raw): {same_raw.mean():.4f} ({same_raw.sum()} / {len(df)})")

    album_norm = df["album"].astype(str).str.strip().str.lower()
    album_name_norm = df["album_name"].astype(str).str.strip().str.lower()
    same_norm = (album_norm == album_name_norm)
    print(f"Normalized match: {same_norm.mean():.4f} ({same_norm.sum()} / {len(df)})")

    # Esempi di righe discordanti
    diff_mask = (~same_norm) & (~df["album"].isna()) & (~df["album_name"].isna())
    df_diff = df.loc[diff_mask, ["album", "album_name"]].head(30)
    print("\nSome examples (normalized):")
    display(df_diff)

    # -------------------------
    # Altair bar plot con colori custom
    # -------------------------
    stats = pd.DataFrame({
        "type": [
            "raw_match",
            "normalized_match",
            "normalized_mismatch"
        ],
        "count": [
            int(same_raw.sum()),
            int(same_norm.sum()),
            int(len(df) - same_norm.sum())
        ]
    })

    chart = (
        alt.Chart(stats)
        .mark_bar()
        .encode(
            x=alt.X("type:N", title="Comparison type"),
            y=alt.Y("count:Q", title="Number of rows"),
            color=alt.Color(
                "type:N",
                title="Type",
                scale=alt.Scale(
                    domain=["raw_match", "normalized_match", "normalized_mismatch"],
                    range=["#ce93d8", "#9c27b0", "#ba68c8"]
                )
            ),
            tooltip=["type", "count"]
        )
        .properties(
            width=400,
            height=300,
            title="Album vs Album Name - Match overview"
        )
    )

    chart.display()

else:
    print("There aren't `album` and `album_name`.")


##### Consistency with id_album

For each `album id`, we check: 
- how many distinct `album name` entries
- how many distinct `album` entries
This helps us understand which column is the most stable as a textual representation.

In [ ]:
if "id_album" in df.columns:
    # --- Distribuzione album_name per id_album ---
    if "album_name" in df.columns:
        album_name_per_id = df.groupby("id_album")["album_name"].nunique()
        dist_album_name = (
            album_name_per_id.value_counts()
            .sort_index()
            .to_frame(name="n_id_album")
            .rename_axis("n_unique_album_name")
            .reset_index()
        )
        display(dist_album_name.head(10))

    # --- Distribuzione album per id_album ---
    if "album" in df.columns:
        album_per_id = df.groupby("id_album")["album"].nunique()
        dist_album = (
            album_per_id.value_counts()
            .sort_index()
            .to_frame(name="n_id_album")
            .rename_axis("n_unique_album")
            .reset_index()
        )
        display(dist_album.head(10))

    # --- Casi sospetti: stesso id_album con più nomi diversi ---
    if "album_name" in df.columns:
        suspicious_albumname = album_name_per_id[album_name_per_id > 1]
        print(f"\n`id_album` with more than one `album_name`: {len(suspicious_albumname)}")
        display(
            df[df["id_album"].isin(suspicious_albumname.index)]
            [["id_album", "album_name"]]
            .drop_duplicates()
            .head(30)
        )

    if "album" in df.columns:
        suspicious_album = album_per_id[album_per_id > 1]
        print(f"\n`id_album` with more than one `album`: {len(suspicious_album)}")
        display(
            df[df["id_album"].isin(suspicious_album.index)]
            [["id_album", "album"]]
            .drop_duplicates()
            .head(30)
        )
else:
    print("Missing id_album")


After this checks, we found that:
- album_name is reliable: almost all album_ids have only one album_name (only 5 duplicates), so you use that as the official album name.

- album is dirty: many album_ids have multiple different values ​​(or NaN), so you treat it as the original album_raw and don't use it for keys, groupbys, or deduplications.

This information will be taken in consideration in the next sections.

#### Title and Full_title

I am going to do the same made with album and album_name.
The goal is to understand if the two coloums have the same information. In order to do it we analize:
1. quality and consistency of `title` and `full_title`

In [ ]:
def aggressive_clean(s):
    """
    Normalizza una stringa per un confronto affidabile:
    1. Converte in stringa e minuscolo.
    2. Rimuove caratteri invisibili (es. zero-width space).
    3. Sostituisce spazi non-secabili (NBSP) con spazi normali.
    4. Normalizza tutti i tipi di apostrofi (curvi, accenti) in un apostrofo dritto.
    5. NUOVO: Normalizza tutti i tipi di virgolette (curve) in virgolette dritte.
    6. Collassa tutti i tipi di whitespace (spazi, tab, \n) in un singolo spazio.
    7. Rimuove spazi bianchi all'inizio e alla fine.
    """
    if pd.isna(s):
        return ''
    s = str(s).lower()
    s = s.replace('\u200b', '')
    s = s.replace(u'\xa0', ' ')

    s = re.sub(r'[\’\‘`´]', "'", s)

    s = re.sub(r'[\“\”\„]', '"', s)

    s = re.sub(r'\s+', ' ', s)

    return s.strip()

print("--- Containment Check: 'full_title' vs. ('title' 'artist' 'featuring') ---")
print("Running with APOSTROPHE + QUOTATION MARK NORMALIZATION...")

# --- 2. Definisci la funzione di controllo di contenimento ---
def check_containment(row):
    """
    Controlla se le versioni pulite di title, artist e features
    sono tutte contenute nella versione pulita di full_title.
    """
    clean_full = aggressive_clean(row['full_title'])
    clean_title = aggressive_clean(row['title'])
    clean_artist = aggressive_clean(row['name_artist'])
    # clean_features = aggressive_clean(row['featured_artists'])

    if not clean_full or not clean_title or not clean_artist:
        return pd.NA

    if clean_title not in clean_full:
        return False

    if clean_artist not in clean_full:
        return False


    return True


print("Applying containment check to all rows...")
containment_mask = df.apply(check_containment, axis=1)

valid_checks = containment_mask.dropna()
num_mismatches = (valid_checks == False).sum()
num_matches = (valid_checks == True).sum()
num_skipped = containment_mask.isna().sum()

print(f"\nTotal rows checked: {len(df)}")
print(f"Rows skipped (missing title, artist, or full_title): {num_skipped}")
print(f"Rows where 'full_title' (pulito) CONTAINS all info: {num_matches}")
print(f"Rows where 'full_title' (pulito) IS MISSING some info: {num_mismatches}")

if num_mismatches > 0:
    print("\n--- Inspecting Mismatches (Rows missing info) ---")
    print("Queste sono le righe in cui 'full_title' (pulito) NON contiene")
    print("il 'title', 'name_artist'  (tutti puliti).")

    mismatch_df = df[containment_mask == False].copy()

    mismatch_df['check_full_title'] = mismatch_df['full_title'].apply(aggressive_clean)
    mismatch_df['check_title'] = mismatch_df['title'].apply(aggressive_clean)
    mismatch_df['check_artist'] = mismatch_df['name_artist'].apply(aggressive_clean)


    display(mismatch_df[[
        'full_title',
        'check_full_title',
        'check_title',
        'check_artist',
    ]].head(20))
else:
    print("\nSUCCESS: All 'full_title' rows (with valid data) contain the required info.")


##### Consistency with id

In [ ]:
if "id" in df.columns and {"title", "full_title"}.issubset(df.columns):
    title_per_id = df.groupby("id")["title"].nunique()
    full_per_id = df.groupby("id")["full_title"].nunique()

    display(
        title_per_id.value_counts()
        .sort_index()
        .to_frame("n_id")
        .rename_axis("n_unique_title")
        .reset_index()
        .head(10)
    )

    display(
        full_per_id.value_counts()
        .sort_index()
        .to_frame("n_id")
        .rename_axis("n_unique_full_title")
        .reset_index()
        .head(10)
    )

    # id con problemi: più di un title o più di un full_title
    bad_id = title_per_id[title_per_id > 1].index.union(
        full_per_id[full_per_id > 1].index
    )

    display(
        df[df["id"].isin(bad_id)]
        [["id", "title", "full_title"]]
        .drop_duplicates()
        .sort_values(["id", "title", "full_title"])
        .head(50)
    )


They track the same information. We will take full_name.

#### Name and Name_artist
Objective:
- Verify name consistency for each `id_artist`
- Determine whether `name_artist` is redundant with `name`

In [ ]:
if "id_artist" in df.columns:
    # name_artist per id_artist
    if "name_artist" in df.columns:
        na_per_id = df.groupby("id_artist")["name_artist"].nunique()
        dist_na = (
            na_per_id.value_counts()
            .sort_index()
            .to_frame("n_id_artist")
            .rename_axis("n_unique_name_artist")
            .reset_index()
        )
        display(dist_na.head(10))

    # name per id_artist (dopo merge con artist.csv)
    if "name" in df.columns:
        n_per_id = df.groupby("id_artist")["name"].nunique()
        dist_n = (
            n_per_id.value_counts()
            .sort_index()
            .to_frame("n_id_artist")
            .rename_axis("n_unique_name")
            .reset_index()
        )
        display(dist_n.head(10))

In [ ]:
display(df[['name', 'name_artist']])

Let's check which one is more stable.

In [ ]:
if "id_artist" in df.columns:
    # --- Distribuzione `name` per id_artist ---
    if "name" in df.columns:
        name_per_id = df.groupby("id_artist")["name"].nunique()
        dist_name = (
            name_per_id.value_counts()
            .sort_index()
            .to_frame(name="n_id_artist")
            .rename_axis("n_unique_name")
            .reset_index()
        )
        display(dist_name.head(10))

    # --- Distribuzione `name_artist` per id_artist ---
    if "name_artist" in df.columns:
        name_artist_per_id = df.groupby("id_artist")["name_artist"].nunique()
        dist_name_artist = (
            name_artist_per_id.value_counts()
            .sort_index()
            .to_frame(name="n_id_artist")
            .rename_axis("n_unique_name_artist")
            .reset_index()
        )
        display(dist_name_artist.head(10))

    # --- Casi sospetti: più nomi per lo stesso id_artist ---
    if "name" in df.columns:
        suspicious_name = name_per_id[name_per_id > 1]
        print(f"\n`id_artist` with more than one `name`: {len(suspicious_name)}")
        display(
            df[df["id_artist"].isin(suspicious_name.index)]
            [["id_artist", "name"]]
            .drop_duplicates()
            .head(30)
        )

    if "name_artist" in df.columns:
        suspicious_name_artist = name_artist_per_id[name_artist_per_id > 1]
        print(f"\n`id_artist` with more than one `name_artist`: {len(suspicious_name_artist)}")
        display(
            df[df["id_artist"].isin(suspicious_name_artist.index)]
            [["id_artist", "name_artist"]]
            .drop_duplicates()
            .head(30)
        )
else:
    print("Missing id_artist")

This to coloums are identical. We will remove artist_name.

#### Name vs Primary Artists

In [ ]:
# Normalize both columns for comparison (temporary variables)
name_clean = df['name'].str.lower().str.strip()
primary_clean = df['primary_artist'].str.lower().str.strip()

# Boolean Series for matches (not added to df)
matches = name_clean == primary_clean

# Show mismatched rows
mismatches = df[~matches][['name', 'primary_artist']]

print(f"Total mismatches: {len(mismatches)}")
display(mismatches)

### Outliers Detection


In [ ]:
# Numerical Feature Definition

# List of key numerical columns to analyze
skewed_features = [
    'tokens_per_sent', 'avg_token_per_clause', 'duration_ms', 'stats_pageviews', 'swear_EN', 'char_per_tok', 'swear_IT', 'bpm', 'n_sentences', 'n_tokens', 'rolloff', 'zcr', 'lexical_density', 'flatness'
]

simetric_features =[
    'pitch', 'centroid', 'spectral_complexity', 'loudness', 'flux', 'rms'
]


I removed the following variables from the list, the statistical analysis of outliers (IQR/Z-Score) is semantically wrong for them: disc_number, track_number, Month, Day

In [ ]:
n_cols = 3
n_rows = -(-len(simetric_features) // n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(simetric_features):
    sns.boxplot(x=df[col], ax=axes[i], orient='h', color="skyblue")
    axes[i].set_title(col, fontsize=14)
    axes[i].set_xlabel("")

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("Box Plot Analysis for Outlier Detection", fontsize=20, y=1.03)
plt.tight_layout()
plt.show()

All six plots show distributions that appear relatively symmetric. The median line within each box is positioned near the center of the box (the Interquartile Range, or IQR), indicating that the 50th percentile is roughly equidistant from the 25th (Q1) and 75th (Q3) percentiles.

Every feature have the presence of candidate outliers.

Outlier Distribution:

pitch, centroid, flux, and rms all show outliers on both the lower (left) and upper (right) ends of their distributions.

spectral_complexity and loudness appear to have outliers almost exclusively on the high end (right side).

This visual inspection suggests that while the central tendency of these features is symmetrically distributed, a small number of records possess extremely high and/or low values that fall outside the main data cluster.

In [ ]:
n_cols = 3
n_rows = -(-len(skewed_features) // n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(skewed_features):
    sns.boxplot(x=df[col], ax=axes[i], orient='h', color="skyblue")
    axes[i].set_title(col, fontsize=14)
    axes[i].set_xlabel("")

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("Box Plot Analysis for Outlier Detection", fontsize=20, y=1.03)
plt.tight_layout()
plt.show()

* **Extreme Skewness:** For many features, particularly `stats_pageviews`, `swear_EN`, `swear_IT`, `tokens_per_sent`, and `avg_token_per_clause`, the main "box" is extremely compressed and pushed to the far left. This visually confirms a strong positive skew (right-tailed distribution).

* **Vast Number of Outliers:** The most prominent characteristic is the dense cloud of outliers extending far to the right for these positively skewed features. This indicates that a large number of records have values significantly higher than the main cluster of data.

* **Specific Cases:**
    * **Positively Skewed:** `stats_pageviews` and `swear_EN` are the most extreme examples, where the box is barely visible, and the plot is dominated by a long stream of high-end outliers.
    * **Negatively Skewed:** `flatness` is the clear exception, showing the opposite pattern. Its box is compressed to the far right, with a long tail of outliers on the low end (left side), confirming its negative skew.
    * **Mixed Outliers:** Features like `duration_ms`, `n_sentences`, and `lexical_density` show outliers on *both* sides, though the high-end outliers are generally more numerous or extreme.

The visual analysis shows that these features are heavily skewed and contain a large quantity of extreme values. Simply removing all these outliers would lead to massive data loss.

In [ ]:
# Analisi Statistica: Metodo IQR (per variabili asimmetriche)
outlier_data = []

for col in skewed_features:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    low_outliers = df[df[col] < lower_bound]
    high_outliers = df[df[col] > upper_bound]

    outlier_data.append({
        'variable': col,
        'High Outliers (Above)': len(high_outliers),
        'Low Outliers (Below)': len(low_outliers)
    })

    print(f"Variable '{col}':")
    print(f"  IQR Limits: [{lower_bound:.2f}, {upper_bound:.2f}]")
    print(f"  Found {len(low_outliers)} outliers below the limit.")
    print(f"  Found {len(high_outliers)} outliers above the limit.")

In [ ]:
if outlier_data:
    df_outliers = pd.DataFrame(outlier_data)

    # Calculate the total outliers for sorting
    df_outliers['total_outliers'] = df_outliers['High Outliers (Above)'] + df_outliers['Low Outliers (Below)']

    df_outliers = df_outliers.sort_values(by='total_outliers', ascending=False)

    sort_order = df_outliers['variable'].tolist()

    df_melted = df_outliers.melt(
        id_vars=['variable', 'total_outliers'],
        value_vars=['High Outliers (Above)', 'Low Outliers (Below)'],
        var_name='Outlier Type',
        value_name='Number of Outliers'
    )

    domain_ = ["High Outliers (Above)", "Low Outliers (Below)"]
    range_ = ["#d36ba8", "#8cbcd9"]

    base = alt.Chart(df_melted).mark_bar().encode(
        y=alt.Y('Number of Outliers', title='Number of Outliers Detected'),

        color=alt.Color('Outlier Type',
                        legend=alt.Legend(title='Outlier Type'),
                        scale=alt.Scale(domain=domain_, range=range_)),

        tooltip=['variable', 'Outlier Type', 'Number of Outliers']

    ).properties(
        title='Outlier Count (IQR Method) for Skewed Variables'
    )

    chart = base.encode(
        x=alt.X('Outlier Type', axis=None),
        column=alt.Column('variable',
                          sort=sort_order,
                          header=alt.Header(
                              title='Variable',
                              titleOrient="bottom",
                              labelOrient="bottom",
                              labelAngle=-45,
                              labelAlign='right',
                              labelBaseline='middle',
                              labelPadding=5
                          ))
    ).interactive()

    display(chart)

In [ ]:
# Analisi Statistica: Metodo Z-Score (per variabili simmetriche)
outlier_data_z = []
z_threshold = 3

for col in simetric_features:
    mean = df[col].mean()
    std = df[col].std()

    lower_bound_z = mean - (z_threshold * std)
    upper_bound_z = mean + (z_threshold * std)

    low_outliers_z = df[df[col] < lower_bound_z]
    high_outliers_z = df[df[col] > upper_bound_z]

    outlier_data_z.append({
        'variable': col,
        'High Outliers (Above)': len(high_outliers_z),
        'Low Outliers (Below)': len(low_outliers_z)
    })

    print(f"Variable '{col}':")
    print(f"  Z-Score Limits (threshold={z_threshold}): [{lower_bound_z:.2f}, {upper_bound_z:.2f}]")
    print(f"  Found {len(low_outliers_z)} outliers below the limit.")
    print(f"  Found {len(high_outliers_z)} outliers above the limit.")

The IQR statistical analysis confirms that **all asymmetric features contain a significant number of outliers**, as clearly visualized in the summary bar chart.

* **Massive Outlier Counts:** The most striking case is `swear_EN`, with 2,740 high-end outliers. This is a statistical artifact: its IQR is `[0.00, 0.00]`, meaning any track with even one English swear word is flagged. `swear_IT` (746), `avg_token_per_clause` (605), and `flatness` (507) also show a very high volume of outliers, making simple removal impossible.

* **One-Sided Distributions:**
    * `stats_pageviews`, `swear_EN`, and `swear_IT` only have **high-end outliers**. This perfectly matches their positive skew (right-tail) and identifies "hit songs" or lyrically extreme tracks.
    * `flatness` is the only feature with exclusively **low-end outliers**, confirming its negative skew (left-tail).

* **Two-Sided Distributions:** Most features, including `duration_ms`, `n_sentences`, `lexical_density`, and `char_per_tok`, show a significant number of outliers on **both sides**. This implies the presence of errors or extreme values at both ends (e.g., for `duration_ms`, this likely includes both "skit" tracks and very long songs).

* **Isolated Outliers:** `bpm` is a special case, showing **only one high-end outlier**. This is almost certainly an data-entry error (e.g., `bpm = 900`) and will be simple to inspect and correct.

In [ ]:
# Converti la lista in un DataFrame
if outlier_data_z:
    df_outliers_z = pd.DataFrame(outlier_data_z)

    df_outliers_z['total_outliers'] = df_outliers_z['High Outliers (Above)'] + df_outliers_z['Low Outliers (Below)']

    df_outliers_z = df_outliers_z.sort_values(by='total_outliers', ascending=False)

    sort_order_z = df_outliers_z['variable'].tolist()

    df_melted_z = df_outliers_z.melt(
        id_vars=['variable', 'total_outliers'],
        value_vars=['High Outliers (Above)', 'Low Outliers (Below)'],
        var_name='Outlier Type',
        value_name='Number of Outliers'
    )

    domain_z = ["High Outliers (Above)", "Low Outliers (Below)"]
    range_z = ["#d36ba8", "#8cbcd9"]

    base_z = alt.Chart(df_melted_z).mark_bar().encode(
        y=alt.Y('Number of Outliers', title='Number of Outliers Detected'),
        color=alt.Color('Outlier Type',
                        legend=alt.Legend(title='Outlier Type'),
                        scale=alt.Scale(domain=domain_z, range=range_z)),
        tooltip=['variable', 'Outlier Type', 'Number of Outliers']
    ).properties(
        title='Outlier Count (Z-Score Method) for Symmetric Variables'
    )

    chart_z = base_z.encode(
        x=alt.X('Outlier Type', axis=None),
        column=alt.Column('variable',
                          sort=sort_order_z,
                          header=alt.Header(
                              title='Variable',
                              titleOrient="bottom",
                              labelOrient="bottom",
                              labelAngle=-45,
                              labelAlign='right',
                              labelBaseline='middle',
                              labelPadding=5
                          ))
    ).interactive()

    display(chart_z)


* **Quantification of Outliers:** The analysis provides precise counts for these extreme values. `pitch` emerges as the feature with the most outliers (68 total), while `spectral_complexity` has the fewest (22). `centroid` (57) and `flux` (49) also show a notable number of outliers.

* **Distribution of Outliers:**
    * For most features (`pitch`, `centroid`, `spectral_complexity`, `flux`, and `rms`), the outliers are **distributed on both the high and low ends**. This indicates that there are tracks with values that are exceptionally high *and* exceptionally low for these audio characteristics.
    * The most significant exception is `loudness`, which has 27 identified outliers, all of which are **exclusively on the high side** (above the 50.17 limit). This statistical result perfectly matches its box plot, which only showed an upper tail.

This analysis confirms that while these features are symmetrically distributed, a small number of extreme values are present. These outliers are likely real but rare data points (e.g., unusually loud or high-pitched songs) that must be handled before clustering to prevent them from skewing the model.

### Features Inspection

In this phase, we inspect each feature to identify values that fall outside the expected domain, violate logical constraints, or appear inconsistent with the semantics of the variable. These anomalies may indicate data entry errors, corrupted records, incorrect parsing, or values that require transformation before further analysis.

#### Artists Names
This code groups the dataset by artist name to count how many songs each artist has, then displays the total number of unique artists and sorts them by song count. The analysis shows that there are 104 unique artists in the dataset. Among them, Mondo Marcio, Guè Pequeno, and Gemitaiz are the most prolific, each with over 300 songs. Other highly represented artists include Bassi Maestro, Fabri Fibra, and Vacca, each contributing more than 250 songs. On the other hand, a few artists such as O Zulù, Joey Funboy, and Hindaco have only a handful of tracks. Overall, the distribution highlights a few artists dominating the dataset while many others have significantly fewer songs.

In [ ]:
print(df.shape)

# 1. Conteggio brani per artista
artist_song_counts = df.groupby('name').size()

# 2. Series -> DataFrame
artist_counts_df = (
    artist_song_counts
    .reset_index(name='song_count')
    .sort_values(by='song_count', ascending=False)
)

print(f"\nTotal number of unique artists: {len(artist_counts_df)}")
print("Unique Artists and Their Song Count:")
print(artist_counts_df)

# 3. Bar chart orizzontale in Altair
bar = alt.Chart(artist_counts_df).mark_bar().encode(
    y=alt.Y('name:N',
            sort='-x',
            title='Artist'),
    x=alt.X('song_count:Q',
            title='Number of Songs'),
    color=alt.Color(
        'song_count:Q',
        scale=alt.Scale(range=["#e1bee7", "#ce93d8", "#ba68c8", "#9c27b0"]),
        legend=None
    ),
    tooltip=[
        alt.Tooltip('name:N', title='Artist'),
        alt.Tooltip('song_count:Q', title='Number of Songs')
    ]
).properties(
    title='Number of Songs per Artist',
    width=600,
    height={'step': 16}  # altezza dinamica in base al numero di artisti
)

# 4. Etichette a destra di ogni barra
text = bar.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text='song_count:Q'
)

(bar + text)


##### Unique artist names in alphabetical order

In [ ]:
# Assuming your DataFrame is called df and the artist column is 'name'
unique_artists = sorted(df['name'].dropna().unique())
artists_unique=[]
# Display the results
for artist in unique_artists:
    artists_unique.append(artist)
dtale .show(artists_unique)

##### Checking  whether name_artist and name are the same

The values in the two columns generally match, referring to the same artist, but the name column provides a clearer and more standardized version of the artist’s name (e.g., “M¥SS KETA” instead of “miss keta”). Since both represent the same entity but name is formatted more accurately, we will retain the name column and drop name_artist for clarity and consistency.

In [ ]:
# Create a new column that checks if they match (case-insensitive and stripped)
df["same_artist_name"] = (
    df["name_artist"].str.strip().str.lower() == df["name"].str.strip().str.lower()
)

# See mismatches
mismatched = df[~df["same_artist_name"]]

print("Mismatched rows:")
dtale.show(mismatched[["name_artist", "name"]])


#### Artists' Gender

In [ ]:
# --- Gender distribution ---
gender_counts = df['gender'].value_counts(dropna=False)
gender_percent = (gender_counts / gender_counts.sum()) * 100

# --- Create bar chart ---
plt.figure(figsize=(6, 4))
bars = plt.bar(gender_counts.index.astype(str), gender_counts.values, color='skyblue')

plt.title('Artist Gender Distribution')
plt.xlabel('Gender')
plt.ylabel('Number of Artists')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# --- Add percentage labels on each bar ---
for bar, pct in zip(bars, gender_percent):
    plt.text(
        bar.get_x() + bar.get_width() / 2,  # X position (center)
        bar.get_height(),                   # Y position (top of bar)
        f"{pct:.1f}%",                      # Label text
        ha='center', va='bottom', fontsize=10, fontweight='bold'
    )

plt.tight_layout()
plt.show()

# --- Table of each artist with their gender ---
artist_gender_table = df[['name', 'gender']].drop_duplicates().sort_values(by='name')
dtale.show(artist_gender_table)

print(df['gender'].value_counts())

#### Artists Description
The folowing code counts how many times each unique description appears in the dataset. This helps identify which artist descriptions are the most common or repeated, showing patterns such as groups of artists sharing the same description or potential duplicates.

The results show how the most frequent artist descriptions in the dataset. Most entries describe Italian rappers, producers, or singer-songwriters, reflecting that the dataset mainly focuses on Italian music artists.

For instance, “gruppo musicale italiano” (Italian music group) appears 620 times, making it the most common description.

Interestingly, there are also some non-musical or unrelated entries, like “dio indiano della distruzione e della trasformazione” (Indian god of destruction and transformation) or “tipo di barca a vela usata nel XVIII e XIX secolo” (type of sailing ship used in the 18th–19th century). These seem to be data errors.

We also noticed an entry labeled “gruppo musicale canadese” (Canadian music group). Upon checking, this description is incorrectly assigned to the Italian rapper Priestess. Further research revealed a mix-up with a Canadian band that shares the same name. This confusion becomes evident when comparing the active_start year in the dataset, which matches that of the Canadian group rather than the Italian artist.

In [ ]:
df['description'].value_counts()


##### Identifying groups in the dataset

This filter identifies all artists whose description includes the word "gruppo", which typically refers to musical groups or bands. The resulting list contains 7 well-known Italian music groups, such as 99 Posse, Articolo 31, Club Dogo, Colle Der Fomento, Cor Veleno, Dark Polo Gang, and Sottotono. There is an outlier group, it is canadian. 

In [ ]:
# Filter rows where 'description' contains 'gruppo'
artists_with_gruppo = (
    df[df['description'].str.contains('gruppo', case=False, na=False)]
[['name','description','birth_date','active_start']]
    .drop_duplicates(subset=['name'])
    .sort_values(by='name')
)

print("Artists with 'grupoo' in their description:",artists_with_gruppo.shape )
display(artists_with_gruppo)


#### Artist's BirthDate

##### Birthdate Column Type
The column is object. We are going to  to see the values that cannot be converted to DateTime directly.
Converting to DateTime is needed to inspect the values easier

Based on the values that could not be converted to datetime, we found that the birth_date column contained several invalid entries, such as URLs (e.g., "http://www.wikidata.org/.well-known/genid/...") instead of actual dates. Since these values do not represent meaningful or recoverable information, there is nothing worth preserving. Therefore, we are going to apply the pd.to_datetime(errors='coerce') function directly, allowing all invalid entries to be converted to NaT.

In [ ]:
date_cols = [ 'birth_date']
# --- Check date columns ---
for col in date_cols:
    original = df[col].copy()
    converted = pd.to_datetime(original, errors='coerce')
    non_convertible = original[original.notna() & converted.isna()]
    
    print(f"\nColumn '{col}'  entries that cannot be converted to datetime:")
    if not non_convertible.empty:
        for idx, val in non_convertible.items():
            print(f"Row {idx}: {val}")
    else:
        print("All non-missing entries can be converted to datetime.")
    print('----------------------------------------------------------------')
    
    
# coverting to dateTime   
date_cols = ['birth_date',  ]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')  # convert to datetime, invalid dates become NaT


df.info()


##### Distribution of artist birth_date

This code extracts each artist’s birth year and groups them by decade (e.g., 1960s, 1970s, 1980s, etc.) to analyze how artists are distributed over time. It calculates the percentage of artists born in each decade and visualizes it with a bar chart. The results show that most artists were born between the 1980s and 1990s, indicating that the majority belong to the Millennial generation, while fewer artists were born in the 1960s or after 2000.

It also shows the histogram of the birth year

In [ ]:
# Count missing values in 'birthdate' column
missing_birthdates = df['birth_date'].isna().sum()

print(f"Number of missing values in 'birthdate' column: {missing_birthdates}")

functions.plot_birth_decades(df, "Distribution of Artists' Birth Years  before Cleaning",'Percentage of Unique Artists by Decade of Birth before cleaning')

# Confirm df is unchanged
print(df.columns)


##### Distribution of artist ages
The results show the distribution of unique artists’ ages in the dataset, ranging from 22 to 58 years old. Most artists fall between their late 20s and mid-40s, with small peaks around ages 32, 36, and 46, each having between 6 and 7 artists. Younger artists under 25 and older ones above 50 are less represented. Overall, the majority of artists are in their thirties and early forties, reflecting the typical active and productive age range in the music industry.

In [ ]:

functions.plot_artist_ages(df,"Number of Unique Artists by Age (Before Cleaning)")


##### Artists with no birthdate
This code shows the number of the artist that doesn't have a birthdate. They are 32.

In [ ]:

missing_birth_artists = df[df['birth_date'].isna()]['name'].drop_duplicates()
print(f"Number of artists with missing birth date: {missing_birth_artists.shape[0]}")

print("Artists with missing birth date:")
print(missing_birth_artists.tolist())

##### Artists Names and their ages
This code generates a table showing each unique artist and their corresponding age, calculated from their birth date. Artists are listed from oldest to youngest, highlighting ages from 23 to 58 in this dataset. 

In [ ]:

# --- Calculate age without adding column to df ---
import pandas as pd

def display_artist_ages(df, title="Unique Artists and Their Age"):
    """
    Display a table of unique artists and their ages.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing at least 'name' and 'birth_date' columns.
        title (str): Custom title for the output.
    """
    # --- Compute current age in years ---
    today = pd.Timestamp.today()
    artist_age = (today - df['birth_date']).dt.days // 365

    # --- Create temporary DataFrame with unique artists and their age ---
    artist_age_df = pd.DataFrame({
        'name': df['name'],
        'age': artist_age
    }).drop_duplicates().sort_values(by='age', ascending=False)

    # --- Display the table ---
    print(title + ":")
    display(artist_age_df.style.background_gradient(cmap='coolwarm'))

    # --- Total number of unique artists ---
    print(f"\nTotal number of unique artists: {artist_age_df['name'].nunique()}")

display_artist_ages(df, title="Artists Age Overview Before filling null Values and Cleaning")


#### Active start

##### Active_Start DataType Column
The column is object. We are going to  to see the values that cannot be converted to DateTime directly.
Converting to DateTime is needed to inspect the values easier

We realized that all non-missing entries are  already in a valid date format, so they are going to be  successfully converted to datetime without any issues.

In [ ]:
date_cols = [ 'active_start']
# --- Check date columns ---
for col in date_cols:
    original = df[col].copy()
    converted = pd.to_datetime(original, errors='coerce')
    non_convertible = original[original.notna() & converted.isna()]
    
    print(f"\nColumn '{col}'  entries that cannot be converted to datetime:")
    if not non_convertible.empty:
        for idx, val in non_convertible.items():
            print(f"Row {idx}: {val}")
    else:
        print("All non-missing entries can be converted to datetime.")
    print('----------------------------------------------------------------')
    
#Converting to dateTime
date_cols = [ 'active_start', ]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')  # convert to datetime, invalid dates become NaT


df.info()


##### Percentage of Artists by Active Start Decade
The dataset contains 4,601 missing values in the active_start column, meaning a significant number of artists have no recorded career start date. Considering only unique artists, the distribution across decades shows that the 1990s (32%), 2000s (30%), and 2010s (32%) were the most common periods for artists to begin their careers, indicating a fairly even spread among these decades. Earlier decades like the 1980s (4%) and the 2020s (2%) have much fewer entries, likely reflecting fewer documented artists or incomplete data for those periods.

In [ ]:
print(f"Number of missing values in 'active_start': {df['active_start'].isna().sum()}")
functions.plot_active_start_decades(df,'Percentage of Unique Artists by Active Start Decade before Cleaning')


##### Ages of artists when they started their career
The distribution of ages of unique artists when they started their careers shows that most began between 17 and 22 years old, which is reasonable. However, there are outliers, such as one artist listed as starting at age 1 and another at age 10, which clearly do not make sense. These anomalous values indicate potential data errors, and we need to investigate these specific cases to determine the best way to clean or correct the dataset.

In [ ]:
functions.plot_age_at_career_start(df,'Age of unique Artists When They Started Their Career Before Cleaning')


##### Ckecking artist whose age was 10, 13, 27 when they started their career

Among the unique artists, several had unusual ages at career start.  Nesli (age 10) had incorrect active_start date, while Salmo (age 13) and Mudimbi (age 27) were correct. 

In [ ]:
# lavoriamo su una copia per sicurezza
tmp = df.copy()

# assicuriamoci che siano datetime
tmp['birth_date'] = pd.to_datetime(tmp['birth_date'], errors='coerce')
tmp['active_start'] = pd.to_datetime(tmp['active_start'], errors='coerce')

# calcola age at career start (in anni)
ages = tmp['active_start'].dt.year - tmp['birth_date'].dt.year

# filtra righe con età specifiche
mask = ages.isin([ 10, 13, 27])
outliers = tmp[mask].copy()

# tieni solo artisti unici per nome
unique_outliers = outliers.drop_duplicates(subset=['name']).copy()

# ricalcola l'età solo su questi (così è allineata)
unique_outliers['age_at_start'] = (
    unique_outliers['active_start'].dt.year
    - unique_outliers['birth_date'].dt.year
)

print("Unique artists with age  10, 13, 27 at career start:")
print(unique_outliers[['name', 'birth_date', 'active_start', 'age_at_start']])

##### Artists with no active start date
This code shows the number of the artist that doesn't have an active start date. They are 54 out 104.

In [ ]:
# --- Filter rows where 'active_start' is missing ---
missing_active_start = df[df['active_start'].isna()]

# --- Compute the earliest full release date (from year, month, day) per artist ---
release_dates = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')
album_dates = pd.to_datetime(df['album_release_date'], errors='coerce')

# Group by artist and get earliest song date and earliest album release date
earliest_dates = (
    df.assign(_release_date=release_dates, _album_date=album_dates)
      .groupby('name', as_index=False)
      .agg({'_release_date': 'min', '_album_date': 'min'})
      .rename(columns={'_release_date': 'earliest_song_date', '_album_date': 'earliest_album_date'})
)

# --- Merge with artists missing 'active_start' ---
artists_missing_active = (
    missing_active_start[['name', 'active_start', 'birth_date']]
    .drop_duplicates()
    .merge(earliest_dates, on='name', how='left')
    .sort_values(by='name')
)

# --- Print the result ---
print("Artists without 'active_start' information :")
print(artists_missing_active.to_string(index=False))

# --- Optional count ---
print(f"\nTotal number of unique artists missing 'active_start': {artists_missing_active['name'].nunique()}")


####  Albums and Album Release Date

##### Album release Date DataType Column

The column is object. We are going to  to see the values that cannot be converted to DateTime directly.
Converting to DateTime is needed to inspect the values easier

We realized that all non-missing entries are  already in a valid date format, so they are going to be  successfully converted to datetime without any issues.

Looking at the values in the album_release_date column that could not be converted to datetime, we noticed that many of them were just years (e.g., "2004"). If we used pd.to_datetime(errors='coerce') directly, these entries would have been turned into NaT. However, we wanted to keep this information by assigning a default month and day — the first day of the year.

- Instead of converting the column directly, we applied a cleaning function that:

- Detected values that were only a year (e.g., "2004") and changed them to a full date ("2004-01-01").

- Kept valid full dates (e.g., "2021-04-09") unchanged.

- Left missing values as they are.

- Finally, converted everything into proper datetime format for consistency.

In [ ]:
date_cols = ['album_release_date']
# --- Check date columns ---
for col in date_cols:
    original = df[col].copy()
    converted = pd.to_datetime(original, errors='coerce')
    non_convertible = original[original.notna() & converted.isna()]
    
    print(f"\nColumn '{col}'  entries that cannot be converted to datetime:")
    if not non_convertible.empty:
        for idx, val in non_convertible.items():
            print(f"Row {idx}: {val}")
    else:
        print("All non-missing entries can be converted to datetime.")
    print('----------------------------------------------------------------')
    
# Converting to DateTime
def fix_year_only_dates(val):
    """
    If the value looks like a 4-digit year, convert it to 'YYYY-01-01'.
    Otherwise, return the original value.
    """
    if pd.isna(val):
        return val
    val_str = str(val).strip()
    if re.fullmatch(r'\d{4}', val_str):
        return f"{val_str}-01-01"
    return val_str

# Apply to album_release_date
df['album_release_date'] = df['album_release_date'].apply(fix_year_only_dates)

# Convert to datetime
df['album_release_date'] = pd.to_datetime(df['album_release_date'], errors='coerce')

df.info()

##### Unique albums with their release dates

In [ ]:


def show_unique_albums(df, album_col='album_name', artist_col='name', date_col='album_release_date'):
    """
    Displays all unique albums with their artist and release date using D-Tale.
    Does not modify the original dataset.
    
    Parameters:
    - df: pandas DataFrame
    - album_col: column containing album names
    - artist_col: column containing artist names
    - date_col: column containing album release dates
    
    Returns:
    - unique_albums: DataFrame with unique albums and their details
    """
    # --- Extract unique albums ---
    unique_albums = df.drop_duplicates(subset=[album_col])[['id_album',album_col, artist_col, date_col,'album_type','album_image','disc_number']]
    
    # --- Print summary ---
    print(f"Number of unique albums: {len(unique_albums)}")
    
    # --- Show in D-Tale ---
    return dtale.show(unique_albums)

show_unique_albums(df)


##### Count Unique albums per artist

In [ ]:

def plot_albums_per_artist(df, top_n=200, artist_col='name', album_col='album_name'):
    """
    Displays and plots how many unique albums each artist has.
    Does not modify the dataset.
    
    Parameters:
    - df: pandas DataFrame
    - top_n: number of top artists to display in the plot
    - artist_col: column containing artist names
    - album_col: column containing album names
    """
    # --- Count unique albums per artist ---
    artist_album_counts = (
        df.drop_duplicates(subset=[artist_col, album_col])
          .groupby(artist_col)[album_col]
          .count()
          .reset_index(name='album_count')
          .sort_values(by='album_count', ascending=False)
    )

    # --- Display summary ---
    print(f"Total unique artists: {artist_album_counts.shape[0]}")
    print("Artists by number of albums:")
    print(artist_album_counts.head(top_n))

    # --- Plot ---
    plt.figure(figsize=(15, 20))
    top_artists = artist_album_counts.head(top_n)
    sns.barplot(
        data=top_artists,
        y=artist_col,
        x='album_count',
    
    )

    plt.title("Artists by Number of Unique Albums", fontsize=16, pad=15)
    plt.xlabel("Number of Albums", fontsize=12)
    plt.ylabel("Artist", fontsize=12)

    # --- Add labels ---
    for i, val in enumerate(top_artists['album_count']):
        plt.text(val + 0.1, i, str(val), va='center', fontsize=9)

    sns.despine()
    plt.tight_layout()
    plt.show()

    return artist_album_counts

artist_album_counts = plot_albums_per_artist(df, )


##### Album release decade distribution (unique albums only)

In [ ]:
def plot_unique_album_release_distribution(df, album_col='album_name', date_col='album_release_date',
                                           title="Percentage of Unique Albums by Release Decade"):
    """
    Plots the percentage of unique albums by their release decade (bar plot)
    and a histogram of unique album release years.
    Does not modify the original dataset.
    """
    # --- Keep only unique albums ---
    unique_albums = df.drop_duplicates(subset=[album_col]).copy()

    # --- Extract album release years ---
    album_years = pd.to_datetime(unique_albums[date_col], errors='coerce').dt.year.dropna()

    if album_years.empty:
        print(f"No valid album release years found in column '{date_col}'.")
        return

    # --- Define decade bins ---
    start = int(album_years.min() // 10 * 10)
    end = int(album_years.max() // 10 * 10 + 10)
    bins = list(range(start, end + 10, 10))
    labels = [f"{b}s" for b in bins[:-1]]

    # --- Group by decade ---
    decade_groups = pd.cut(album_years, bins=bins, labels=labels, right=False)

    # --- Calculate percentages ---
    group_percent = decade_groups.value_counts(normalize=True).sort_index() * 100
    group_df = pd.DataFrame({'decade': group_percent.index, 'percent': group_percent.values})

    print(title)
    print(group_df)

    # --- Plot bar chart and histogram side by side ---
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # --- Bar plot ---
    sns.barplot(data=group_df, x='decade', y='percent', hue='decade',
                palette='mako', legend=False, ax=axes[0])
    for i, val in enumerate(group_df['percent']):
        axes[0].text(i, val + 0.5, f"{val:.2f}%", ha='center', fontsize=10)

    axes[0].set_title(title, fontsize=16, pad=15)
    axes[0].set_xlabel("Album Release Decade", fontsize=12)
    axes[0].set_ylabel("Percentage (%)", fontsize=12)
    sns.despine(ax=axes[0])

    # --- Histogram of release years ---
    sns.histplot(album_years, bins=20, kde=True, color='skyblue', ax=axes[1])
    axes[1].set_title("Distribution of Unique Album Release Years", fontsize=16, pad=15)
    axes[1].set_xlabel("Release Year", fontsize=12)
    axes[1].set_ylabel("Count", fontsize=12)
    sns.despine(ax=axes[1])

    plt.tight_layout()
    plt.show()
plot_unique_album_release_distribution(df)


##### Album release date Describtion

In [ ]:
print(df['album_release_date'].describe())

##### Number of Tracks per Album

In [ ]:

def plot_tracks_per_album(df, album_col='album_name', title="Tracks per top 20 Unique Album "):
    """
    Counts how many tracks exist for each unique album and plots:
    
    Bar chart of track counts per album
    """
    # --- Drop missing album names ---
    df_valid = df.dropna(subset=[album_col]).copy()

    # --- Count tracks per album ---
    track_counts = df_valid[album_col].value_counts().sort_values(ascending=False)
    track_df = track_counts.reset_index()
    track_df.columns = [album_col, 'track_count']

    # --- Print summary ---
    print(f"Total unique albums: {len(track_df)}")
    print("\nAlbums by Number of Tracks:")
    print(track_df)


    # ---  Bar chart (each album with its track count) ---
    plt.figure(figsize=(12, 8))
    sns.barplot(
        data=track_df.head(20),
        x='track_count',
        y=album_col,
    )
    plt.title(title, fontsize=16, pad=15)
    plt.xlabel("Number of Tracks", fontsize=12)
    plt.ylabel("Album Name", fontsize=12)
    plt.tight_layout()
    plt.show()

    return track_df

album_track_counts = plot_tracks_per_album(df)


##### Fetch Albums from 50s till 80s

This code identify and analyze albums released between the 1950s and 1980s, summarizing their release years, album types, artist names, and song titles. The goal was to check the accuracy of these albums because they represent the least values in the distribution, so there is a high chance there are error.

There are 24 albums from the 50s till the 80s in the dataset.

Upon inspection, we found that the album release years were mostly accurate — 21 out of 24 albums matched their verified release dates from official sources such as Discogs and Wikipedia. Only a few albums, like Amico È (Inno dell’amicizia) and Dentro e Fuori, had minor differences of about one or two years.

When comparing the albums with the listed artists, most entries turned out to be incorrectly matched. Many classic or international albums, such as Wish You Were Here (Pink Floyd), Rattle and Hum (U2), Babylon By Bus (Bob Marley & The Wailers), and Come Dancing with the Kinks (The Kinks), were mistakenly paired with modern Italian artists like Lazza, Tedua, Baby K, and Gemitaiz.

Most Importantly, when analyzing the albums and their corresponding songs (full_title), none of the listed songs actually belonged to the albums they were associated with. The tracks were all modern works by contemporary Italian artists, whereas the albums were classic releases from entirely different time periods and genres.



In [ ]:
def get_albums_by_decades(
    df,
    start_decade=1950,
    end_decade=1980,
    date_col='album_release_date',
    album_col='album_name',
    track_col='full_title',
    birth_col='birth_date'
):
    """
    Returns albums released between given decades (inclusive),
    along with their album type, artist name, birth date,
    track titles, and a flag if the artist was born after the album release.
    """

    # --- Make a safe copy and ensure proper types ---
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df[birth_col] = pd.to_datetime(df[birth_col], errors='coerce')

    # --- Extract release year ---
    df['release_year'] = df[date_col].dt.year

    # --- Filter albums released in the given range (make a copy to avoid warnings) ---
    filtered = df[
        (df['release_year'] >= start_decade) &
        (df['release_year'] < end_decade + 10)
    ].copy()  # ✅ prevents SettingWithCopyWarning

    # --- Determine if any artist was born after the album release ---
    filtered['artist_born_after_album'] = filtered.apply(
        lambda row: (
            pd.notna(row[birth_col]) and
            pd.notna(row['release_year']) and
            row[birth_col].year > row['release_year']
        ), axis=1
    )

    # --- Group by album and collect information ---
    grouped = (
        filtered.groupby(album_col, dropna=False)
        .agg({
            'release_year': 'first',
            'album_type': lambda x: list(pd.unique(x)),
            'name': lambda x: list(pd.unique(x)),
            birth_col: lambda x: list(pd.unique(x)),
            track_col: lambda x: list(pd.unique(x)),
            'artist_born_after_album': 'any'  ,
            'disc_number':lambda x: list(pd.unique(x)),
            'track_number':lambda x: list(pd.unique(x)),
        })
        .reset_index()
    )

    # --- Display and return ---
    print(f"Albums released between {start_decade}s and {end_decade}s: {grouped[album_col].nunique()} unique albums")
    display(grouped[[album_col, 'release_year', 'album_type', 'name', birth_col, track_col, 'artist_born_after_album','disc_number','track_number']])

    return grouped


# --- Example usage ---
albums_50s_80s = get_albums_by_decades(df, 1950, 1980)
dtale.show(albums_50s_80s)


##### Number of Tracks per Album

In [ ]:
def plot_tracks_per_album(df, album_col='album_name', title="Tracks per top 20 Unique Album "):
    """
    Counts how many tracks exist for each unique album and plots:

    Bar chart of track counts per album
    """
    # --- Drop missing album names ---
    df_valid = df.dropna(subset=[album_col]).copy()

    # --- Count tracks per album ---
    track_counts = df_valid[album_col].value_counts().sort_values(ascending=False)
    track_df = track_counts.reset_index()
    track_df.columns = [album_col, 'track_count']

    # --- Print summary ---
    print(f"Total unique albums: {len(track_df)}")
    print("\nAlbums by Number of Tracks:")
    print(track_df)


    # ---  Bar chart (each album with its track count) ---
    plt.figure(figsize=(12, 8))
    sns.barplot(
        data=track_df.head(20),
        x='track_count',
        y=album_col,
    )
    plt.title(title, fontsize=16, pad=15)
    plt.xlabel("Number of Tracks", fontsize=12)
    plt.ylabel("Album Name", fontsize=12)
    plt.tight_layout()
    plt.show()

    return track_df

album_track_counts = plot_tracks_per_album(df)

##### Distribution of Unique Albums by Type

In [ ]:
def plot_unique_album_types(df, album_col='album_name', type_col='album_type',
                            title="Distribution of Unique Albums by Type"):
    """
    Shows the count and percentage of each album type,
    considering only unique albums (by album name).
    """
    # --- Keep only unique albums ---
    unique_albums = df.drop_duplicates(subset=[album_col]).copy()

    # --- Drop missing album types ---
    unique_albums = unique_albums.dropna(subset=[type_col])

    # --- Count occurrences ---
    type_counts = unique_albums[type_col].value_counts().sort_values(ascending=False)
    type_percent = (type_counts / type_counts.sum() * 100).round(2)

    # --- Combine results into a DataFrame ---
    summary_df = pd.DataFrame({
        'count': type_counts,
        'percent': type_percent
    })

    print("Unique Album Types Summary:\n")
    print(summary_df)

    # --- Plot ---
    plt.figure(figsize=(8, 6))
    sns.barplot(x=summary_df.index, y=summary_df['count'], )

    # Add count and percentage labels
    for i, (count, percent) in enumerate(zip(summary_df['count'], summary_df['percent'])):
        plt.text(i, count + 0.5, f"{count} ({percent}%)", ha='center', fontsize=10)

    plt.title(title, fontsize=16, pad=15)
    plt.xlabel("Album Type", fontsize=12)
    plt.ylabel("Number of Unique Albums", fontsize=12)
    sns.despine()
    plt.tight_layout()
    plt.show()

    return summary_df
plot_unique_album_types(df)


##### Albums with Names but without a release date

In [ ]:
def find_albums_missing_release_date(df, album_col='album_name', artist_col='name', date_col='album_release_date'):
    """
    Finds and lists  albums that have a name but no release date.
    """
    missing = df[df[album_col].notna() & df[date_col].isna()][[album_col, artist_col]]
    print(f"Number of  albums with a name but without release date: {len(missing)}")
    display(missing)
    return missing

missing_albums = find_albums_missing_release_date(df)


##### Tracks without albums

In [ ]:
def find_tracks_without_album(df, track_col='full_title', album_col='album_name', artist_col='name'):
    """
    Lists tracks that do not have an album assigned.
    """
    missing_tracks = df[df[album_col].isna()][[track_col, artist_col,album_col,'album_release_date','album_type']].drop_duplicates()
    print(f"Number of tracks without an album: {len(missing_tracks)}")
    display(missing_tracks['full_title'])
    return missing_tracks

# Usage
tracks_without_album = find_tracks_without_album(df)
dtale.show(tracks_without_album)

#####  Albums that appear with multiple different artist names

In [ ]:
def find_albums_with_multiple_artists(
    df, 
    album_col='album_name', 
    artist_col='name', 
    feature_col='name_artist'
):
    """
    Finds albums that appear with multiple different artist names.
    Also lists the different values of another feature (e.g., 'name_artist') for each album.
    """
    # Exclude rows with missing album names
    df_filtered = df[df[album_col].notna()]

    # Group by album and collect unique artist names and feature values
    grouped = (
        df_filtered
        .groupby(album_col)
        .agg({
            artist_col: lambda x: list(pd.unique(x)),
            feature_col: lambda x: list(pd.unique(x)),
            'featured_artists' : lambda x: list(pd.unique(x)),
            'album_type':lambda x: list(pd.unique(x)),
            'primary_artist':lambda x: list(pd.unique(x)),
        })
        .reset_index()
    )

    # Keep only albums linked to more than one unique artist
    inconsistent_albums = grouped[grouped[artist_col].apply(len) > 1]

    # Rename columns for clarity
    inconsistent_albums = inconsistent_albums.rename(columns={
        artist_col: 'Artist Name from Artists Dataset',
        feature_col: f'Artist Name from Tracks Dataset',
        'featured_artists': 'featured_artists',
         'primary_artist':'primary_artist'
    })

    print(f"Number of albums with multiple artist names: {len(inconsistent_albums)}")
    display(inconsistent_albums.head(20))

    return inconsistent_albums

# Usage
albums_with_multiple_artists = find_albums_with_multiple_artists(df)


##### Albums that have multiple different release dates 
The analysis identified several albums with multiple recorded release dates. There are 43 albums with different release date. We are going to pick the newest release date in Fixing Errors part.

In [ ]:
def find_albums_with_multiple_release_dates(
    df, 
    album_col='album_name', 
    date_col='album_release_date',
    album_type_col='album_type'
):
    """
    Finds albums with multiple different release dates and lists the dates,
    also showing the corresponding album type.
    """
    # Exclude null album names
    df_filtered = df[df[album_col].notna()]

    # Group by album and aggregate unique release dates and album types
    grouped = (
        df_filtered
        .groupby(album_col)
        .agg({
            date_col: lambda x: list(pd.unique(x)),
            album_type_col: lambda x: list(pd.unique(x)),
            'name':lambda x: list(pd.unique(x)),
            'language':lambda x: list(pd.unique(x))
        })
        .reset_index()
    )

    # Keep only albums with more than one unique release date
    inconsistent_albums = grouped[grouped[date_col].apply(len) > 1].copy()

    # Rename for clarity
    inconsistent_albums = inconsistent_albums.rename(columns={
        date_col: 'release_dates',
        album_type_col: 'album_types',
        'name':'name',
        'language':'language'
    })

    print(f"Number of albums with multiple release dates: {len(inconsistent_albums)}")
    display(inconsistent_albums)

    return inconsistent_albums

# Usage
albums_with_inconsistent_dates = find_albums_with_multiple_release_dates(df)


##### Checking for Albums released before artist's birth

There are 23 albums release before the artist has born. After verifying these 23 albums, the albums themselves were wrongly assigned to unrelated artists and songs.

In [ ]:
def find_albums_before_birth(df):
    """
    Find albums released before the artist's birth date.
    """
    album_before_birth = df[df['album_release_date'] < df['birth_date']]

    print(f"Albums released before artist's birth: {len(album_before_birth)}")
    return(album_before_birth[['full_title','album_name', 'album_release_date', 'birth_date', 'name','disc_number','track_number']])

albums_before_birth=find_albums_before_birth(df)
dtale.show(albums_before_birth)

##### Checking for album released before career start

In [ ]:
def find_albums_before_active_start(df):
    """
    Find albums released before the artist's active date.
    """
    album_before_active_start = df[df['album_release_date'] < df['active_start']]

    print(f"Albums released before artist's active start date: {len(album_before_active_start)}")
    return(album_before_active_start[['full_title','album_name', 'album_release_date','active_start', 'birth_date', 'name']])

album_before_active_start=find_albums_before_active_start(df)
display(album_before_active_start)

#### Disk Number and Track Number

In [ ]:
disc_counts = df['disc_number'].value_counts().sort_index()

plt.figure(figsize=(8, 5))
disc_counts.plot(kind='bar')

plt.title('Distribution of Disc Numbers')
plt.xlabel('Disc Number')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


##### Check for missing disc_number or track_number
All entries with null values in either the disc_number or track_number columns were found to belong to albums where these fields are consistently missing across all tracks

In [ ]:
missing_disc_track = df.loc[
    df['disc_number'].isna() | df['track_number'].isna(),
    ['album_name', 'album_type', 'full_title']
]

print(f"Missing disc_number or track_number: {len(missing_disc_track)} rows")
dtale.show(missing_disc_track)


##### Checking for Non numeric values 

In [ ]:
# Check for non-numeric or invalid values in disc_number and track_number (ignoring nulls)
invalid_disc = df[
    df['disc_number'].notna() & ~df['disc_number'].apply(lambda x: float(x).is_integer())
]

invalid_track = df[
    df['track_number'].notna() & ~df['track_number'].apply(lambda x: float(x).is_integer())
]

print(f"Invalid disc_number entries (excluding nulls): {len(invalid_disc)}")
print(invalid_disc[['album_name', 'album_type', 'full_title', 'disc_number', 'track_number']])

print(f"Invalid track_number entries (excluding nulls): {len(invalid_track)}")
print(invalid_track[['album_name', 'album_type', 'full_title', 'disc_number', 'track_number']])


##### Tracks Numbered Outside Expected Ranges

In [ ]:
# Check for track numbers outside expected range (e.g., <=0 or unusually high)
invalid_track_range = df[(df['track_number'] <= 0) | (df['track_number'] > 99)]
print(f"Tracks with invalid numbering: {len(invalid_track_range)}")
print(invalid_track_range)
print('---------------------------------------------------')
print(df['track_number'].value_counts())


##### Albums and their disk number and track number

In [ ]:
# Group albums but keep duplicates
albums_info = (
    df.groupby('album_name')[['disc_number', 'track_number', 'full_title', 'name']]
    .apply(lambda x: x.sort_values(['disc_number', 'track_number']))
    .reset_index()
)

# Display results
for album in albums_info['album_name'].unique():
    subset = albums_info[albums_info['album_name'] == album]
    print(f"\n🎵 Album: {album}")
    for _, row in subset.iterrows():
        print(f"  Disc {int(row['disc_number'])}, Track {int(row['track_number'])}: "
              f"{row['full_title']} — {row['name']}")


##### Track Number Sequence Validation
A total of 2,600 discs were found to have track sequence issues, indicating widespread inconsistencies in the ordering or numbering of songs within albums. These issues include missing or duplicate track numbers, sequences that don’t start at 1, or tracks that are out of order.
 
The following issues indicate potential problems in the dataset, as they suggest that some albums may have incomplete or inconsistent track listings:

Track numbers skip values, implying that some tracks might be missing.

The track sequence does not start at 1, suggesting incomplete metadata or incorrect ordering.

For this analysis, we will focus only on the duplicate track numbers, since these represent clear inconsistencies that can be directly identified and corrected.

In [ ]:
def find_track_sequence_issues(df, album_col='album_name', disc_col='disc_number', track_col='track_number'):
    """
       Finds albums/discs with track numbering issues:
    - Missing numbers (gaps)
    - Duplicate track numbers
    - Track numbers not starting at 1
    - Non-integer or invalid values
    """
    
    issues = []
    numberOfDuplicates =0
    # Group by album and disc
    for (album, disc), group in df.groupby([album_col, disc_col]):
        tracks = group[track_col].dropna().tolist()
        if not tracks:
            continue
        

        
           # Ensure numeric comparison
        try:
            tracks = sorted([int(x) for x in tracks])
        except ValueError:
            issues.append({
                'album_name': album,
                'disc_number': disc,
                'issue': 'Non-numeric track numbers',
                'track_numbers': group[track_col].unique().tolist()
            })
            continue
        
        # Detect duplicates
        if len(tracks) != len(set(tracks)):
            issues.append({
                'album_name': album,
                'disc_number': disc,
                'issue': 'Duplicate track numbers',
                'track_numbers': tracks
            })
            numberOfDuplicates =numberOfDuplicates+1

        # Detect missing numbers
        expected = list(range(1, max(tracks) + 1))
        missing = sorted(set(expected) - set(tracks))
        if missing:
            issues.append({
                'album_name': album,
                'disc_number': disc,
                'issue': f'Missing track numbers: {missing}',
                'track_numbers': tracks
            })

        # Detect wrong start
        if tracks[0] != 1:
            issues.append({
                'album_name': album,
                'disc_number': disc,
                'issue': f'Track sequence does not start at 1 (starts at {tracks[0]})',
                'track_numbers': tracks
            })


    issues_df = pd.DataFrame(issues)
    print(f"Number of discs with sequence issues: {len(issues_df)}")
    print('Number of Disks that have Duplicates number of tracks ',numberOfDuplicates)
    return issues_df
track_issues = find_track_sequence_issues(df)
dtale.show(track_issues)


In [ ]:
# identifying tracks number duplicates
duplicates = df[df.duplicated(subset=['album_name', 'disc_number', 'track_number'], keep=False)]
print(f"🎵 Found {len(duplicates)} duplicate track entries.")
dtale.show(duplicates[['album_name', 'disc_number', 'track_number', 'full_title', 'name']])


#### Duration

In [ ]:
print('missing values',df['duration_ms'].isna().sum())
plt.hist(df['duration_ms'].dropna(), bins=50)
plt.xlabel("Duration (ms)")
plt.ylabel("Frequency")
plt.title("Distribution of Track Duration (ms)")
plt.show()


In [ ]:
# Create duration in minutes
df['duration_minutes'] = df['duration_ms'] / 60000

# Select and sort
song_durations = (
    df[['full_title', 'title', 'duration_ms', 'duration_minutes']]
    .sort_values(by='duration_minutes', ascending=False)
)

# Display the result
song_durations

#### Track Year 


##### Track Year DataType Column 
The column is Object.  We are going to  to see the values that cannot be converted to Numbers directly.
Converting to Number is needed to inspect the values easier.

Inspecting the values in the year column, we observed that while most entries were numerical, some contained unexpected or non-numeric characters. To handle this, we converted the column directly to a numeric type using pd.to_numeric() with the errors='coerce' parameter, which automatically transforms any invalid or non-numeric values into NaN.

In [ ]:
numeric_cols = ['year']

# --- Check numeric columns ---
for col in numeric_cols:
    original = df[col].copy()
    converted = pd.to_numeric(original, errors='coerce')
    non_convertible = original[original.notna() & converted.isna()]
    
    print(f"\nColumn '{col}'  entries that cannot be converted to numeric:")
    if not non_convertible.empty:
        for idx, val in non_convertible.items():
            print(f"Row {idx}: {val}")
    else:
        print("All non-missing entries can be converted to numeric.")
    print('----------------------------------------------------------------')
    
#Converting to numbers    
    
df['year'] = pd.to_numeric(df['year'], errors='coerce') 
df.info()

##### Track Year Distribution

Looking at the distribution of values in the track year in the previous section, we notice some entries before 1950 and after 2025, which don’t make much sense. Therefore, we will investigate these cases further to understand the cause and decide how to correct them.


Result:
From the plots, we can see that more than half of the songs and albums were released between 2000 and 2020, indicating that most of the data comes from the recent two decades

In [ ]:
def plot_year_distribution_by_decade(df, title, year_col='year'):
    """
    Plot the percentage distribution of entries by decade based on a given year column.
    Does not modify the dataset.
    """

    # --- Convert 'year' to numeric safely ---
    years = pd.to_numeric(df[year_col], errors='coerce').dropna()

    # --- Define decade bins (e.g., 1960s, 1970s, ..., 2020s) ---
    start = int((years.min() // 10) * 10)
    end = int((years.max() // 10) * 10 + 10)
    bins = list(range(start, end + 10, 10))
    labels = [f"{b}s" for b in bins[:-1]]

    # --- Categorize years into decades ---
    decade_groups = pd.cut(years, bins=bins, labels=labels, right=False)

    # --- Calculate percentage per decade ---
    group_percent = decade_groups.value_counts(normalize=True).sort_index() * 100
    group_df = pd.DataFrame({'decade': group_percent.index, 'percent': group_percent.values})

    # --- Plot ---
    plt.figure(figsize=(10, 6))
    sns.barplot(data=group_df, x='decade', y='percent', hue='decade', palette='viridis', legend=False)

    # --- Add percentage labels ---
    for i, val in enumerate(group_df['percent']):
        plt.text(i, val + 0.5, f"{val:.2f}%", ha='center', fontsize=10)

    plt.title(title, fontsize=18, pad=15)
    plt.xlabel("Decade", fontsize=12)
    plt.ylabel("Percentage (%)", fontsize=12)
    sns.despine()
    plt.tight_layout()
    plt.show()
    
plot_year_distribution_by_decade(df, "Percentage of Songs by Decade  before cleaning")


Descriptive Statistics
The summary statistics show that the song release years range from 1900 to 2100, with an average around 2013, indicating some unrealistic future values.

In [ ]:
# For the 'year' column
print(df['year'].describe())  

##### Number of Songs before 1950 and after 2025

In [ ]:
def check_songs_before_1950(tracks):
    """
    Identify and display tracks released before 1950.
    """
    tracks['year'] = pd.to_numeric(tracks['year'], errors='coerce')
    songs_before_1950 = tracks[tracks['year'] < 1950].shape[0]

    print(f"Number of songs before 1950: {songs_before_1950}")

    return tracks[tracks['year'] < 1950][['full_title', 'album_name', 'album_release_date', 'album_type', 'year', 'month', 'day']]
     
   
old_songs =check_songs_before_1950(tracks)
dtale.show(old_songs)

In [ ]:
def check_songs_after_2025(tracks):
    """
    Identify and display tracks released after 2025.
    """
    tracks['year'] = pd.to_numeric(tracks['year'], errors='coerce')
    songs_after_2025 = tracks[tracks['year'] > 2025].shape[0]

    print('-----------------------------------------------')
    print(f"Number of songs after 2025: {songs_after_2025}")

    return tracks[tracks['year'] > 2025][['full_title', 'album_name', 'album_release_date', 'album_type', 'year', 'month', 'day']]
 
future_songs =check_songs_after_2025(tracks)
dtale.show(future_songs)

#### Inconsistency with years

##### Checking if the active_start date is earlier than the artist’s birth_date

In [ ]:
def find_invalid_active_start(df):
    """
    Find artists whose active_start date is earlier than their birth date.
    """
    df['birth_date'] = pd.to_datetime(df['birth_date'], errors='coerce')
    df['active_start'] = pd.to_datetime(df['active_start'], errors='coerce')

    invalid_dates = df[df['active_start'] < df['birth_date']]

    print(f"Found {len(invalid_dates)} artists with 'active_start' earlier than 'birth_date'.")
    display(invalid_dates[['id_artist', 'name_artist', 'birth_date', 'active_start']])
find_invalid_active_start(df)

##### Checking for tracks released (year) before the artist’s career started (active start)

In [ ]:
def find_tracks_before_career_start(df):
    """
    Find tracks released before the artist's career start.
    """
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['active_start'] = pd.to_datetime(df['active_start'], errors='coerce')

    inconsistency = df[
        (df['year'].notna()) &
        (df['active_start'].notna()) &
        (df['year'] < df['active_start'].dt.year)
    ]

    print(f"⚠️ Number of records where a song was released before the artist's career start: {len(inconsistency)}")

    return inconsistency[['name', 'full_title', 'year', 'active_start', 'album_release_date']].sort_values(by='name')
    
tracks_beforer_career=find_tracks_before_career_start(df)
dtale.show(tracks_beforer_career)

##### Checking for tracks released before the artist’s birth

In [ ]:
def find_tracks_before_birth(df):
    """
    Find tracks released before the artist's birth date.
    """
    tracks_before_birth = df[df['year'] < df['birth_date'].dt.year]

    print(f"Number of tracks released before artist's birth: {len(tracks_before_birth)}")
    return (tracks_before_birth[['full_title', 'year', 'birth_date', 'album_release_date', 'name_artist']])
tracks_before_birth =find_tracks_before_birth(df)
dtale.show(tracks_before_birth)

##### Checking for album released before career start

In [ ]:
def find_albums_before_career(df):
    """
    Find albums released before the artist's career start date.
    """
    album_before_career = df[df['album_release_date'] < df['active_start']]

    print(f"Albums released before artist's career start: {len(album_before_career)}")
    return (album_before_career[['full_title', 'album_release_date', 'active_start', 'name_artist']])
albums_before_career =find_albums_before_career(df)
dtale.show(albums_before_career)

##### Checking for Albums released before artist's birth

In [ ]:
def find_albums_before_birth(df):
    """
    Find albums released before the artist's birth date.
    """
    album_before_birth = df[df['album_release_date'] < df['birth_date']]

    print(f"Albums released before artist's birth: {len(album_before_birth)}")
    return(album_before_birth[['full_title', 'album_release_date', 'birth_date', 'name_artist']])
albums_before_birth=find_albums_before_birth(df)
dtale.show(albums_before_birth)

##### Checking for Tracks released before album release excluding singles

In [ ]:
def find_tracks_before_album(df):
    """
    Find tracks released before their album release (excluding singles).
    """
    tracks_before_album = df[
        (df['year'] < df['album_release_date'].dt.year) &
        (df['album_type'] != 'single')
    ]

    print(f"Tracks released before the album (excluding singles): {len(tracks_before_album)}")
    return (tracks_before_album[['full_title', 'year', 'album_release_date', 'album_type', 'name_artist']])
tracks_before_album =find_tracks_before_album(df)
dtale.show(tracks_before_album)

####  Artists Location Statistics

##### Inspecting Coordinates and Birthplace

Upon inspecting the geographic data, we found that the vast majority of coordinates correctly referred to each artist’s birth place. However, three rows were inconsistent: their birth places were listed as Almería, Buenos Aires, and Singapore, but the the coordinates either null or refer to province of the artist

To maintain consistency across the dataset, we are going to replace the coordinates in these rows with the correct latitude and longitude of the corresponding birth places.

In [ ]:
df_copy=df.copy()
# Drop rows with no birth_place
df_copy = df_copy.dropna(subset=['birth_place'])

# Keep only the columns we care about
cols = ['birth_place', 'latitude', 'longitude','province','region']

# Drop duplicates so each place appears once (keeping the first lat/lon found)
unique_places = df_copy[cols].drop_duplicates(subset=['birth_place'])

# Sort alphabetically by birth_place (optional)
unique_places = unique_places.sort_values(by='birth_place').reset_index(drop=True)

# Display results
dtale.show(unique_places)


##### Getting null rows

In [ ]:
# Define columns to check
cols_to_check = [
    'birth_place',
    'nationality',
    'province',
    'region',
    'country',
    'latitude',
    'longitude'
]

# Filter rows where any of these columns are null
missing_rows = df[df[cols_to_check].isnull().any(axis=1)]

# Select only artist name + the relevant columns
columns_to_show = ['name'] + cols_to_check
missing_subset = missing_rows[columns_to_show]

# Keep only unique artist names (first occurrence)
unique_missing_subset = missing_subset.drop_duplicates(subset=['name'])

# Show the result in D-Tale
print(unique_missing_subset.shape)
display(unique_missing_subset)


##### Checking Coordinates

In [ ]:
geo_outliers = df[(df['latitude'] < 35.5) | (df['latitude'] > 47.1) |
                  (df['longitude'] < 6.6) | (df['longitude'] > 18.5)]
print(f"Number of Geographic coordinates outside Italy range: {len(geo_outliers)} records")
display(geo_outliers[['name_artist', 'latitude', 'longitude', 'birth_place']].head(10))

##### Artists' Country Values

All the countries have the value of Italia

In [ ]:
# Count the occurrences of each country
country_counts = df['country'].value_counts()

# Plot
plt.figure(figsize=(10, 6))
country_counts.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('Distribution of Artists by Country', fontsize=14, pad=12)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Number of Artists', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


##### Checking if there is an artist his/her country not Italy but his/her coordinates are in Italy

In [ ]:
# Filter rows where country is not Italy and coordinates are present
non_italy_with_coords = df[
    (df['country'].notna()) & 
    (df['country'] != "Italia") & 
    (df['latitude'].notna()) & 
    (df['longitude'].notna())
]

# Count the number of such records
num_records = len(non_italy_with_coords)
print(f"Number of non-Italy records with coordinates: {num_records}")

# Show the records
print(non_italy_with_coords[['country', 'latitude', 'longitude']])


##### Artists Nationality Distribution

Almost all artists are Italian (99.5%), with a small minority from Argentina (0.5%).

In [ ]:
def plot_nationality_distribution(df, column='nationality', top_n=20):
    """
    Plots the percentage distribution of artist nationalities.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataset containing nationality information.
    column : str, optional
        The column name containing nationality data (default = 'nationality').
    top_n : int, optional
        Number of top nationalities to display in the chart (default = 20).
    """

    # Count and calculate percentages
    nat_counts = df[column].value_counts(dropna=True)
    nat_percent = (nat_counts / nat_counts.sum()) * 100
    nat_df = nat_percent.reset_index()
    nat_df.columns = [column, 'percent']

    # Plot setup
    plt.figure(figsize=(10, 8))
    sns.barplot(
        data=nat_df.head(top_n),
        x='percent',
        y=column,
        hue=column,
        palette='crest',
        dodge=False
    )

    plt.title(f"Percentage of Artists by {column.capitalize()}", fontsize=18, pad=15)
    plt.xlabel("Percentage (%)", fontsize=12)
    plt.ylabel(column.capitalize(), fontsize=12)

    # Add percentage labels
    for index, value in enumerate(nat_df.head(top_n)['percent']):
        plt.text(value + 0.5, index, f"{value:.1f}%", va='center', fontsize=9, color='#000000')

    plt.xlim(0, nat_df['percent'].max() + 5)
    sns.despine()
    plt.tight_layout()
    plt.show()

    # Return the computed DataFrame for reference
    return nat_df
# Example usage
nat_df = plot_nationality_distribution(df)


##### Checking if there are artists with Non-Italian Nationality and Italian Coordinates
There is one instance repeated 40 times where the artist has a nationality other than Italian (all Argentinian) but also have italian geographic coordinates.

In [ ]:
def get_non_italy_with_coords(df, nationality_col='nationality',
                              lat_col='latitude', lon_col='longitude'):
    """
    Filters rows where nationality is not 'Italia' and coordinates are present.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataset containing nationality and coordinate columns.
    nationality_col : str, optional
        Column name for nationality (default = 'nationality').
    lat_col : str, optional
        Column name for latitude (default = 'latitude').
    lon_col : str, optional
        Column name for longitude (default = 'longitude').

    Returns
    -------
    pandas.DataFrame
        Filtered DataFrame of non-Italy records with valid coordinates.
    """

    # Filter the data
    non_italy = df[
        (df[nationality_col].notna()) &
        (df[nationality_col].str.lower() != "italia") &
        (df[lat_col].notna()) &
        (df[lon_col].notna())
        ]

    # Count and print summary
    num_records = len(non_italy)
    print(f"Number of non-Italy nationality records with coordinates: {num_records}\n")

    # Show preview of relevant columns
    print(non_italy[[nationality_col, lat_col, lon_col]].head(10))

    return non_italy

get_non_italy_with_coords(df)

##### Distribution of Artist's Birth Places

The majority of artists were born in major Italian cities, with Milano (1,843) and Roma (1,048) being the most frequent birthplaces, indicating a strong concentration of artists from these cultural and economic centers.

Smaller Italian towns such as Senigallia (443), Torino (397), and Avellino (329) also show notable representation, suggesting a widespread national distribution beyond just the biggest cities.

Only a few artists were born outside Italy — such as Buenos Aires (40) and Almería (26) — representing less than 1% of the total, which confirms that the dataset is predominantly composed of Italian-born artists.

In [ ]:
def plot_birthplace_distribution(df, column='birth_place', top_n=None):
    """
    Plots the distribution of a categorical column (default: 'birth_place'),
    showing both counts and percentages above each bar.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataset containing the column to analyze.
    column : str, optional
        The column name to plot (default = 'birth_place').
    top_n : int or None, optional
        Show only the top N most frequent categories (default = all).
    """

    # Count and percentage
    counts = df[column].value_counts()
    if top_n:
        counts = counts.head(top_n)
    percents = (counts / len(df)) * 100

    # Print counts for inspection
    print(counts)

    # Plot
    plt.figure(figsize=(14, 6))
    bars = plt.bar(counts.index, counts.values, color='skyblue')

    # Labels and title
    plt.title(f'Distribution of {column.replace("_", " ").title()}', fontsize=14)
    plt.xlabel(column.replace("_", " ").title())
    plt.ylabel('Count')

    # Add count and percent labels above bars
    for i, (count, percent) in enumerate(zip(counts.values, percents.values)):
        plt.text(i, count + (count * 0.01),
                 f"{count:,}\n({percent:.1f}%)",
                 ha='center', va='bottom', fontsize=6, color='black')

    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
# Plot all birthplaces
plot_birthplace_distribution(df)


##### Checking Birth Place vs Country

This section verifies whether each artist’s birth place matches their country.  We have one instance in which the artist was born in Almería but his country is Italy.

In [ ]:
def check_birthplace_country_mismatch(
        df,
        italian_cities=None,
        foreign_cities_to_country=None,
        birth_col='birth_place',
        country_col='country'
):
    """
    Detect mismatches between birth_place and country columns.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataset containing artist information.
    italian_cities : list, optional
        List of Italian city names to validate (default = predefined list).
    foreign_cities_to_country : dict, optional
        Mapping of known foreign cities to their expected country names.
    birth_col : str, optional
        Column name for birthplace (default = 'birth_place').
    country_col : str, optional
        Column name for country (default = 'country').

    Returns
    -------
    pandas.DataFrame
        DataFrame with a new boolean column 'birth_place_country_mismatch'
        and prints mismatch summary.
    """

    # Default Italian cities list
    if italian_cities is None:
        italian_cities = [
            "Milano", "Roma", "Senigallia", "Torino", "Avellino", "Cagliari", "Salerno",
            "Olbia", "Napoli", "Vimercate", "Vicenza", "Verona", "Scampia", "Nicosia",
            "Sternatia", "Padova", "Grottaglie", "La Spezia", "Scafati", "Nocera Inferiore",
            "Sesto San Giovanni", "Genova", "Alpignano", "Fiumicino", "Treviso", "Bologna",
            "San Siro", "Rho", "Brescia", "Grugliasco", "Reggio Calabria", "Gallarate",
            "Desenzano del Garda", "Pieve Emanuele", "San Benedetto del Tronto", "Firenze",
            "Lodi"
        ]

    # Default foreign city mappings
    if foreign_cities_to_country is None:
        foreign_cities_to_country = {
            "Singapore": "Singapore",
            "Buenos Aires": "Argentina",
            "Almería": "Spagna",
        }

    # Inner function for row-level logic
    def _check(row):
        birth = row.get(birth_col)
        country = row.get(country_col)
        if pd.notna(birth) and pd.notna(country):
            if birth in italian_cities and country != "Italia":
                return True  # mismatch
            elif birth in foreign_cities_to_country:
                if country != foreign_cities_to_country[birth]:
                    return True  # mismatch
        return False  # coherent or missing data

    # Apply to dataframe
    df = df.copy()
    df['birth_place_country_mismatch'] = df.apply(_check, axis=1)

    # Summary
    num_mismatches = df['birth_place_country_mismatch'].sum()
    print(f"Number of birth_place-country mismatches: {num_mismatches}")

    # Show mismatched records
    mismatched = df[df['birth_place_country_mismatch']]
    if not mismatched.empty:
        print("\nMismatched records:")
        display(mismatched[[birth_col, country_col]])
    else:
        print("\n✅ No mismatches found.")

    return df

# Apply to your DataFrame
df_checked = check_birthplace_country_mismatch(df)


##### Birth Place vs Nationality

We have two instance in which the artist was born outside Italy but the nationality is Italian. Chadia rodriguez  was born in Almería and baby k was born in  Singapore.

In [ ]:
def check_birthplace_nationality_mismatch(
        df,
        italian_cities=None,
        foreign_cities_to_nationality=None,
        birth_col='birth_place',
        nationality_col='nationality',
        name_col='name'
):
    """
    Detect mismatches between birthplace and nationality.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataset containing birthplace and nationality columns.
    italian_cities : list, optional
        List of Italian city names (default = predefined Italian cities).
    foreign_cities_to_nationality : dict, optional
        Mapping of known foreign cities to their expected nationality names.
    birth_col : str, optional
        Column name for birthplace (default = 'birth_place').
    nationality_col : str, optional
        Column name for nationality (default = 'nationality').
    name_col : str, optional
        Column name for artist name (default = 'name').

    Returns
    -------
    pandas.DataFrame
        Original DataFrame with an added boolean column:
        'birth_place_nationality_mismatch'
    """

    # Default list of Italian cities
    if italian_cities is None:
        italian_cities = [
            "Milano", "Roma", "Senigallia", "Torino", "Avellino", "Cagliari", "Salerno",
            "Olbia", "Napoli", "Vimercate", "Vicenza", "Verona", "Scampia", "Nicosia",
            "Sternatia", "Padova", "Grottaglie", "La Spezia", "Scafati", "Nocera Inferiore",
            "Sesto San Giovanni", "Genova", "Alpignano", "Fiumicino", "Treviso", "Bologna",
            "San Siro", "Rho", "Brescia", "Grugliasco", "Reggio Calabria", "Gallarate",
            "Desenzano del Garda", "Pieve Emanuele", "San Benedetto del Tronto", "Firenze",
            "Lodi"
        ]

    # Default mapping of foreign cities
    if foreign_cities_to_nationality is None:
        foreign_cities_to_nationality = {
            "Singapore": "Singapore",
            "Buenos Aires": "Argentina",
            "Almería": "Spagna",
        }

    # Define the mismatch check logic
    def _check(row):
        birth = row.get(birth_col)
        nationality = row.get(nationality_col)
        if pd.notna(birth) and pd.notna(nationality):
            if birth in italian_cities and nationality != "Italia":
                return True  # mismatch
            elif birth in foreign_cities_to_nationality:
                if nationality != foreign_cities_to_nationality[birth]:
                    return True  # mismatch
        return False

    # Apply to DataFrame
    df = df.copy()
    df['birth_place_nationality_mismatch'] = df.apply(_check, axis=1)

    # Report
    num_mismatches = df['birth_place_nationality_mismatch'].sum()
    print(f"Number of birth_place-nationality mismatches: {num_mismatches}")

    # Display mismatched records (if any)
    mismatched_records = df[df['birth_place_nationality_mismatch']]
    if not mismatched_records.empty:
        print("\nMismatched records:")
        print(mismatched_records[[name_col, birth_col, nationality_col]].drop_duplicates())
    else:
        print("\n✅ No mismatches found.")

    return df
# Run the function on your DataFrame
df_checked = check_birthplace_nationality_mismatch(df)

##### Distribution of Songs by Province and Region

This code calculates and visualizes the percentage distribution of songs by province and region. It counts occurrences, converts them to percentages, and displays bar charts with labeled values to show which areas have the highest song representation

In [ ]:
def plot_location_distributions(
        df,
        province_col='province',
        region_col='region',
        top_n=20
):
    """
    Plots percentage distributions for province and region columns.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataset containing province and region columns.
    province_col : str, optional
        Column name for province (default = 'province').
    region_col : str, optional
        Column name for region (default = 'region').
    top_n : int, optional
        Number of top entries to display (default = 20).
    """

    # ---------- Province Plot ----------
    province_counts = df[province_col].value_counts()
    province_percent = (province_counts / province_counts.sum()) * 100
    province_df = province_percent.reset_index()
    province_df.columns = [province_col, 'percent']

    print("Provinces:\n", province_counts, "\n")

    plt.figure(figsize=(10, 8))
    sns.barplot(
        data=province_df.head(top_n),
        x='percent',
        y=province_col,
        hue=province_col,
        palette='viridis',
        dodge=False
    )

    plt.title("Percentage of Songs by Province", fontsize=20, pad=15, color="#000000")
    plt.xlabel("Percentage (%)", fontsize=12)
    plt.ylabel("Province", fontsize=12)

    # Add labels
    for index, value in enumerate(province_df.head(top_n)['percent']):
        plt.text(value + 0.5, index, f"{value:.1f}%", va='center', fontsize=9, color='#000000')

    plt.xlim(0, province_df['percent'].max() + 5)
    sns.despine()
    plt.tight_layout()
    plt.show()

    # ---------- Region Plot ----------
    region_counts = df[region_col].value_counts()
    region_percent = (region_counts / region_counts.sum()) * 100
    region_df = region_percent.reset_index()
    region_df.columns = [region_col, 'percent']

    print("Regions:\n", region_counts, "\n")

    plt.figure(figsize=(10, 8))
    sns.barplot(
        data=region_df,
        x='percent',
        y=region_col,
        hue=region_col,
        palette='coolwarm',
        dodge=False
    )

    plt.title("Percentage of Songs by Region", fontsize=20, pad=15, color="#000000")
    plt.xlabel("Percentage (%)", fontsize=12)
    plt.ylabel("Region", fontsize=12)

    for index, value in enumerate(region_df['percent']):
        plt.text(value + 0.5, index, f"{value:.1f}%", va='center', fontsize=9, color='#000000')

    plt.xlim(0, region_df['percent'].max() + 5)
    sns.despine()
    plt.tight_layout()
    plt.show()

    # Return summary DataFrames
    return province_df, region_df
# Example usage
province_df, region_df = plot_location_distributions(df)

##### Province/Region vs Country


In [ ]:
# Example mapping of Italian regions to their provinces (from your data)
region_provinces = {
    "Lombardia": ["Milano", "Monza e della Brianza", "Brescia", "Varese", "Lodi"],
    "Campania": ["Salerno", "Napoli", "Avellino"],
    "Lazio": ["Roma"],
    "Veneto": ["Vicenza", "Verona", "Padova", "Treviso"],
    "Piemonte": ["Torino"],
    "Sardegna": ["Cagliari", "Gallura"],
    "Puglia": ["Lecce", "Taranto"],
    "Liguria": ["Genova", "La Spezia"],
    "Sicilia": ["Enna"],
    "Emilia-Romagna": ["Bologna"],
    "Calabria": ["Reggio Calabria"],
    "Marche": ["Ancona", "Ascoli Piceno"],
    "Toscana": ["Firenze"]
}

# Flatten all Italian provinces for quick lookup
all_italian_provinces = [prov for provs in region_provinces.values() for prov in provs]

# Function to check province/region ↔ country
def check_province_region_country(row):
    if pd.notna(row['country']):
        if pd.notna(row['province']) and row['province'] in all_italian_provinces:
            if row['country'] != "Italia":
                return True  # mismatch
        elif pd.notna(row['region']) and row['region'] in region_provinces.keys():
            if row['country'] != "Italia":
                return True  # mismatch
    return False  # coherent or missing data

# Apply the function
df['province_region_country_mismatch'] = df.apply(check_province_region_country, axis=1)

# Count mismatches
num_mismatches = df['province_region_country_mismatch'].sum()
print(f"Number of province/region-country mismatches: {num_mismatches}")

# Show records with mismatch
mismatched_records = df[df['province_region_country_mismatch']]
print(mismatched_records[['province', 'region', 'country']])


##### Province/Region vs Birth Place

This check compares each artist’s birth_place with the corresponding province and region. Mismatches occur when the province or region does not align with the birth_place. There are 19 uniques cases in which the province/region doesn't match the birth_place.

In [ ]:
def check_birthplace_province_region_mismatch(
        df,
        region_provinces=None,
        birth_col='birth_place',
        province_col='province',
        region_col='region'
):
    """
    Checks for mismatches between a person's birthplace, province, and region.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataset containing birthplace, province, and region columns.
    region_provinces : dict, optional
        Mapping of regions → list of provinces/birthplaces.
        (If None, uses the predefined mapping from your dataset.)
    birth_col : str, optional
        Column name for birthplace (default = 'birth_place').
    province_col : str, optional
        Column name for province (default = 'province').
    region_col : str, optional
        Column name for region (default = 'region').

    Returns
    -------
    pandas.DataFrame
        Original DataFrame with an added boolean column:
        'birth_place_province_region_mismatch'.
    """

    # Default mapping (your provided one)
    if region_provinces is None:
        region_provinces = {
            "Lombardia": ["Milano", "Vimercate", "Sesto San Giovanni", "Alpignano", "Fiumicino",
                          "Brescia", "Grugliasco", "Rho", "Gallarate", "Desenzano del Garda", "Lodi", "San Siro"],
            "Lazio": ["Roma"],
            "Piemonte": ["Torino"],
            "Campania": ["Salerno", "Napoli", "Avellino", "Scafati", "Nocera Inferiore"],
            "Veneto": ["Vicenza", "Verona", "Padova", "Treviso"],
            "Sardegna": ["Cagliari", "Olbia", "Gallura"],
            "Puglia": ["Lecce", "Taranto", "Grottaglie", "Sternatia", "San Benedetto del Tronto"],
            "Liguria": ["Genova", "La Spezia"],
            "Sicilia": ["Enna", "Nicosia"],
            "Emilia-Romagna": ["Bologna"],
            "Calabria": ["Reggio Calabria"],
            "Marche": ["Ancona", "Senigallia", "Ascoli Piceno"],
            "Toscana": ["Firenze", "Scampia", "Padova"]
        }

    # Flatten mapping to get province → region
    province_to_region = {
        prov: reg for reg, provs in region_provinces.items() for prov in provs
    }

    # Row-level check
    def _check(row):
        birth = row.get(birth_col)
        prov = row.get(province_col)
        reg = row.get(region_col)

        if pd.notna(birth) and birth in province_to_region:
            expected_region = province_to_region[birth]

            # Check for mismatched province or region
            if (pd.notna(prov) and prov != birth) or (pd.notna(reg) and reg != expected_region):
                return True
        return False

    # Apply check to DataFrame
    df = df.copy()
    df['birth_place_province_region_mismatch'] = df.apply(_check, axis=1)

    # Report
    num_mismatches = df['birth_place_province_region_mismatch'].sum()
    print(f"Number of birth_place–province/region mismatches: {num_mismatches}")

    mismatched = df[df['birth_place_province_region_mismatch']].drop_duplicates(
        subset=[birth_col, province_col, region_col]
    )
    if not mismatched.empty:
        print("\nMismatched records:")
        print(mismatched[[birth_col, province_col, region_col]])
    else:
        print("\n✅ No mismatches found.")

    return df
# Run the function on your dataset
df_checked = check_birthplace_province_region_mismatch(df)


##### Geographic Distribution of Artists by Province and Region

This analysis aggregates the number of artists by their latitude, longitude, province, and region. The resulting table shows the locations with the highest concentration of artists at the top. For example, Milano (Lombardia) has the most artists with 1,843, followed by Roma (Lazio) with 1,048, and Torino (Piemonte) with 397. The code also generates a map where the size and color of the points reflect the number of artists per location, providing a clear visual of artist density across Italy.

In [ ]:
# Aggregate by latitude and longitude to count number of artists
location_counts = df.groupby(['latitude', 'longitude', 'region', 'province']).size().reset_index(name='num_artists')

# Sort by number of artists descending
location_counts = location_counts.sort_values(by='num_artists', ascending=False)

# Print the sorted table
print(location_counts)

# Define a color scale
color_scale = [(0, 'orange'), (1,'red')]

# Create the scatter map
fig = px.scatter_mapbox(
    location_counts,
    lat="latitude",
    lon="longitude",
    hover_data=["region", "province", "num_artists"],  # show count on hover
    size="num_artists",  # size of marker represents number of artists
    color="num_artists",  # color also shows density
    color_continuous_scale=color_scale,
    zoom=5,
    height=800,
    width=800
)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


#### Popularity

The popularity feature is a key indicator representing the estimated popularity of a track, typically defined in a normalized range between 0 and 100.

In [ ]:
# Check type of popularity

mask_simboli = df['popularity'].astype(str).str.contains(r'%|[MKmk]|views|[A-Za-z]', na=False)

valori_strani2 = (
    df.loc[mask_simboli, 'popularity']
      .dropna()
      .drop_duplicates()
      .sort_values()
)

print(valori_strani2.to_list())




Looking at the values of the  `popularity` column, we noticed that some entries contained **non-numeric characters**, percent signs (`%`), abbreviations like `K` (thousands) or `M` (millions), and words such as `"views"` appended to the numbers.  

Instead of converting the column directly to numeric using pd.to_numeric(errors='coerce'), which would have turned all invalid entries into NaN, we applied a cleaning function to preserve and correctly interpret useful numeric information before conversion. The function:

- Removed non-numeric characters and words like `"views"` and `%`.
- Converted abbreviations (`K → 1,000`, `M → 1,000,000`) to numeric values.
- Extracted the first numeric part if extra text was present.
- Converted the cleaned values to floats, marking any remaining invalid entries as `NaN`.

In [ ]:
def clean_popularity(value):
    if pd.isna(value):
        return None
    value_str = str(value).strip().lower()  # normalize
    
    # Remove common words like 'views'
    value_str = value_str.replace('views','').replace('%','').strip()
    value_str = value_str.lower()  
    # Handle K and M
    multiplier = 1
    if value_str.endswith('k'):
        multiplier = 1_000
        value_str = value_str[:-1]
    elif value_str.endswith('m'):
        multiplier = 1_000_000
        value_str = value_str[:-1]
    
    # Take only first token if words remain
    value_str = value_str.split()[0]
    
    # Try converting to float
    try:
        return (float(value_str) * multiplier)
    except:
        return None  # invalid entries become None/NaN
    
df['popularity'].apply(clean_popularity)

def clean_popularity(value):
    if pd.isna(value):
        return np.nan

    value_str = str(value).strip().lower()

    # togli parole/simboli comuni
    value_str = value_str.replace('views', '').replace('%', '').strip()

    # gestisci suffissi K / M
    multiplier = 1
    if value_str.endswith('k'):
        multiplier = 1_000
        value_str = value_str[:-1]
    elif value_str.endswith('m'):
        multiplier = 1_000_000
        value_str = value_str[:-1]

    # prendi solo il primo token se rimane testo
    value_str = value_str.split()[0]

    try:
        return float(value_str) * multiplier
    except:
        return np.nan

# applica la pulizia
df['popularity'] = df['popularity'].apply(clean_popularity)


df['popularity'] = df['popularity'].astype('float')


In [ ]:
# Missing values count and percentage
missing_pop = df['popularity'].isna().sum()
missing_perc = (missing_pop / len(df)) * 100

print("Missing popularity values:", missing_pop)
print(f"Missing popularity percentage: {missing_perc:.2f}%")


In [ ]:
# Conteggi
low = df[df['popularity'] < 0].shape[0]
high = df[df['popularity'] > 100].shape[0]
total = low + high

# DataFrame per il plot
plot_df = pd.DataFrame({
    "condition": ["popularity < 0", "popularity > 100", "out of range total"],
    "count": [low, high, total]
})

print("Values < 0:", low) 
print("Values > 100:", high) 
print("Total out of range:", low + high)

# Grafico Altair
chart = alt.Chart(plot_df).mark_bar().encode(
    x=alt.X("condition:N", title="Condition"),
    y=alt.Y("count:Q", title="Number of tracks"),
    color=alt.Color(
        "condition:N",
        scale=alt.Scale(range=["#ba68c8", "#e1bee7", "#9c27b0"])  # primi 3 della tua palette
    ),
    tooltip=["condition", "count"]
).properties(
    title="Popularity Out of Range Check",
    width=400,
    height=300
)

chart

In [ ]:
# Summary statistics
df['popularity'].describe()


In [ ]:
# Zero popularity analysis
zero_count = (df['popularity'] == 0).sum()
zero_perc = (zero_count / len(df)) * 100

print("Popularity == 0:", zero_count)
print(f"Percentage of zeros: {zero_perc:.2f}%")


##### Popularity vs Modified Popularity

Goal: to verify whether songs with `modified_popularity = 1` (adjusted popularity scores)  
show a different distribution of `popularity` than unmodified songs.  

We analyze:
- Distribution through boxplots and violin plots  
- Summary statistics  
- Non-parametric test (Mann–Whitney U)  
- Effect size (Cohen’s d)

In [ ]:
# Ensure numeric conversion
df["popularity"] = pd.to_numeric(df["popularity"], errors='coerce')
df["stats_pageviews"] = pd.to_numeric(df.get("stats_pageviews", np.nan), errors='coerce')

# Clean modified_popularity column (it may contain strings like 'True', 'Yes', etc.)
if "modified_popularity" in df.columns:
    df["modified_popularity"] = (
        df['modified_popularity']
        .astype(str).str.strip().str.lower()
        .map({'1':1,'true':1,'yes':1,'y':1,'t':1,'0':0,'false':0,'no':0,'n':0,'f':0})
        .fillna(df['modified_popularity'].apply(lambda x: 1 if x in [1, True] else 0))
        .astype('int64')
    )
else:
    df['modified_popularity'] = 0

Let's check if all the out of range popularity have modified_popularity equal 1

In [ ]:
# Identify rows where popularity is out of the expected range (0–100)
out_of_range = df[(df['popularity'] < 0) | (df['popularity'] > 100)]

print(f"Found {len(out_of_range)} rows with invalid popularity values:\n")
display(out_of_range[['popularity', 'modified_popularity']].head(50))

In [ ]:
colors = ["#e1bee7", "#9c27b0"]

# Violin plot (KDE workaround)
violin = (
    alt.Chart(df)
    .transform_density(
        "popularity",
        as_=["popularity", "density"],
        groupby=["modified_popularity"]
    )
    .mark_area(orient="horizontal")
    .encode(
        y=alt.Y("popularity:Q", title="Popularity"),
        x=alt.X("density:Q", stack="center", title=None),
        color=alt.Color("modified_popularity:N", scale=alt.Scale(range=colors))
    )
    .properties(width=300, title="Violin plot — Popularity by Modified")
)

# Boxplot
box = (
    alt.Chart(df)
    .mark_boxplot(size=70)
    .encode(
        x=alt.X("modified_popularity:N", title="modified_popularity"),
        y=alt.Y("popularity:Q", title="Popularity"),
        color=alt.Color("modified_popularity:N", scale=alt.Scale(range=colors))
    )
    .properties(width=200, title="Boxplot — Popularity by Modified")
)

alt.hconcat(violin, box).resolve_scale(color="shared")

Tracks that show inconsistent popularity across duplicates tend to be more popular and exhibit greater variability in popularity values than tracks without inconsistencies.


#### Language

We analyzed the language attribute to understand how tracks are distributed across different languages and to check whether language labeling is consistent within the dataset.

In [ ]:
# Check type of popularity
print("\nLanguage dtype:")
print(df['language'].dtype)

In [ ]:
# Valori unici presenti nella colonna
df['language'].unique()


In [ ]:
# Conteggio lingue includendo i NaN
lang_counts = df['language'].value_counts(dropna=False).reset_index()
lang_counts.columns = ['language', 'count']

# Plot
alt.Chart(lang_counts).mark_bar().encode(
    x=alt.X('count:Q', title='Number of Tracks'),
    y=alt.Y('language:N', sort='-x', title='Language'),
    color=alt.Color(
        'language:N',
        scale=alt.Scale(range=["#f3e5f5", "#e1bee7", "#ce93d8", "#ba68c8", "#9c27b0"])
    ),
    tooltip=['language', 'count']
).properties(
    title="Tracks per Language (Including NaN)",
    width=600,
    height=300
)


The language attribute initially contained many invalid or inconsistent values (noise, non-ISO codes, and missing entries). After filtering the dataset and keeping only valid language codes, we observed that the majority of tracks are in Italian, with English representing a small secondary portion. This confirms that the dataset is strongly focused on the Italian music market.


In [ ]:
total = len(df)
it_count = len(df[df['language'] == 'it'])
nan_count = len(df[df['language'].isna()])
non_it_count = total - it_count - nan_count

print(f"Total songs: {total}")
print(f"IT songs: {it_count}")
print(f"NaN (unknown language): {nan_count}")
print(f"Non-IT & Non-NaN songs: {non_it_count}")


In [ ]:
display(df[['language', 'lyrics', 'swear_EN', 'swear_IT']].head(50))

#### Swear Words

In [ ]:
display(df[['swear_IT', 'swear_IT_words']].head(50))

In [ ]:
display(df[['swear_EN', 'swear_EN_words']].head(50))

##### Check the counts of swear word IT/EN

To check if there are some errors I am going to use a model to check if the number of swear words is the same of the one reported.

In [ ]:
IT_COUNT_COL = "swear_IT"
EN_COUNT_COL = "swear_EN"
IT_WORDS_COL = "swear_IT_words"
EN_WORDS_COL = "swear_EN_words"

TOKEN_PATTERN = re.compile(r"\b\w+\b", re.UNICODE)

def normalize_token(tok: str) -> str:
    tok = str(tok).lower()
    # compress crazy repetitions: "fuuuuuck" -> "fuuck"
    tok = re.sub(r"(.)\1{2,}", r"\1\1", tok)
    return tok

def parse_swear_list(value):
    """
    Robust parser for swear_*_words.
    Supports:
      - real Python lists: ['cazzo', 'figa']
      - strings that LOOK like lists: "['cazzo', 'figa']"
      - simple strings: "cazzo, figa", "cazzo|figa", "cazzo figa"
    """
    # Case 1: already a list/tuple/set
    if isinstance(value, (list, tuple, set)):
        return [normalize_token(v) for v in value if str(v).strip()]

    # Case 2: not a string -> nothing
    if not isinstance(value, str) or not value.strip():
        return []

    s = value.strip()

    # Case 3: looks like "['a', 'b']" or similar
    if (s[0] in "[(" and s[-1] in "])") or (s.startswith("{") and s.endswith("}")):
        try:
            parsed = ast.literal_eval(s)
            if isinstance(parsed, (list, tuple, set)):
                return [normalize_token(v) for v in parsed if str(v).strip()]
        except Exception:
            # fall back to regex split if literal_eval fails
            pass

    # Case 4: generic split on separators
    parts = re.split(r"[,\|;/\s]+", s)
    return [normalize_token(p) for p in parts if p]

def safe_int(x):
    try:
        if x is None:
            return None
        if isinstance(x, str) and x.strip() == "":
            return None
        return int(x)
    except Exception:
        return None

def count_swears_in_lyrics(lyrics, swear_list):
    """
    Count how many times words in swear_list appear in lyrics.
    Returns (total_count, {word: count})
    """
    if not isinstance(lyrics, str) or not swear_list:
        return 0, {}

    tokens = TOKEN_PATTERN.findall(lyrics.lower())
    norm_tokens = [normalize_token(t) for t in tokens]

    target_set = set(swear_list)
    detail = {}
    for t in norm_tokens:
        if t in target_set:
            detail[t] = detail.get(t, 0) + 1

    total = sum(detail.values())
    return total, detail


it_auto_counts = []
it_auto_details = []
it_matches = []

en_auto_counts = []
en_auto_details = []
en_matches = []

for idx, row in df.iterrows():
    lyrics = row.get("lyrics", "")

    # ----- ITALIAN SWEARS -----
    it_targets = parse_swear_list(row.get(IT_WORDS_COL, ""))
    it_declared = safe_int(row.get(IT_COUNT_COL))
    it_total, it_detail = count_swears_in_lyrics(lyrics, it_targets)

    if it_declared is not None and it_targets:
        it_ok = (it_declared == it_total)
    else:
        it_ok = None

    # ----- ENGLISH SWEARS -----
    en_targets = parse_swear_list(row.get(EN_WORDS_COL, ""))
    en_declared = safe_int(row.get(EN_COUNT_COL))
    en_total, en_detail = count_swears_in_lyrics(lyrics, en_targets)

    if en_declared is not None and en_targets:
        en_ok = (en_declared == en_total)
    else:
        en_ok = None

    # Debug print: only if there is something to check
    if idx % 500 == 0 and (it_targets or en_targets or it_declared not in (None, 0) or en_declared not in (None, 0)):
        print(f"[{idx}] IT targets={it_targets} declared={it_declared} found={it_total} match={it_ok}")
        if it_detail:
            print(f"       IT detail={it_detail}")
        print(f"[{idx}] EN targets={en_targets} declared={en_declared} found={en_total} match={en_ok}")
        if en_detail:
            print(f"       EN detail={en_detail}")

    it_auto_counts.append(it_total)
    it_auto_details.append(it_detail)
    it_matches.append(it_ok)

    en_auto_counts.append(en_total)
    en_auto_details.append(en_detail)
    en_matches.append(en_ok)

df["swear_IT_auto_count"] = it_auto_counts
df["swear_IT_auto_detail"] = it_auto_details
df["swear_IT_match"] = it_matches

df["swear_EN_auto_count"] = en_auto_counts
df["swear_EN_auto_detail"] = en_auto_details
df["swear_EN_match"] = en_matches

# ---- STATS: only rows where we have both declared count AND target words ----

it_valid = df.apply(
    lambda r: (safe_int(r.get(IT_COUNT_COL)) is not None) and bool(parse_swear_list(r.get(IT_WORDS_COL, ""))),
    axis=1
)

en_valid = df.apply(
    lambda r: (safe_int(r.get(EN_COUNT_COL)) is not None) and bool(parse_swear_list(r.get(EN_WORDS_COL, ""))),
    axis=1
)

In [ ]:
print("\n=== ITALIAN SWEAR COUNT CONSISTENCY ===")
print(f"Rows with valid Italian annotations: {it_valid.sum()}")
print(f"Rows with correct Italian counts: {(df.loc[it_valid, 'swear_IT_match'] == True).sum()}")
print(f"Rows with WRONG Italian counts: {(df.loc[it_valid, 'swear_IT_match'] == False).sum()}")

print("\n=== ENGLISH SWEAR COUNT CONSISTENCY ===")
print(f"Rows with valid English annotations: {en_valid.sum()}")
print(f"Rows with correct English counts: {(df.loc[en_valid, 'swear_EN_match'] == True).sum()}")
print(f"Rows with WRONG English counts: {(df.loc[en_valid, 'swear_EN_match'] == False).sum()}")


display(df[['swear_IT', 'swear_EN', 'swear_IT_auto_count', 'swear_EN_auto_count']].head(50))

There are no errors. We can remove all the coloums.

In [ ]:
df.drop(columns=['swear_IT_auto_count'], inplace=True)
df.drop(columns=['swear_IT_auto_detail'], inplace=True)
df.drop(columns=['swear_IT_match'], inplace=True)

df.drop(columns=['swear_EN_auto_count'], inplace=True)
df.drop(columns=['swear_EN_auto_detail'], inplace=True)
df.drop(columns=['swear_EN_match'], inplace=True)


#### Lyrics

In [ ]:
print("Analysis of 'Strange' Short Lyrics ")

# Inspect the 73 'Empty' Lyrics
# These are rows where lyrics is NOT missing, but n_tokens IS missing.
# This happens when lyrics is an empty string "" or similar.
empty_lyrics = df[df['lyrics'].notna() & df['n_tokens'].isna()]

print(f"\nFound {empty_lyrics.shape[0]} rows with 'empty' lyrics (non-NaN lyrics but NaN n_tokens).")
if not empty_lyrics.empty:
    print("These are 'strange values' that need to be cleaned.")

    display(empty_lyrics[['id', 'full_title', 'lyrics', 'n_tokens']].head(20))


In [ ]:
# Inspect 'Very Short' (but not empty) Lyrics
short_lyrics = df[(df['n_tokens'] > 0) & (df['n_tokens'] < 20)]

print(f"\nFound {short_lyrics.shape[0]} songs with very short lyrics.")
if not short_lyrics.empty:
    print("Inspecting these 'strange' short lyrics:")
    display(short_lyrics[['id', 'full_title', 'lyrics', 'n_tokens', 'lexical_density']].sort_values('n_tokens'))

In [ ]:
for index, row in short_lyrics.iterrows():

    print(f"\n==============================================")
    print(f"INDEX (original): {index}")

    # Print the title for context, if present
    if 'full_title' in row:
        print(f"TITLE: {row['full_title']}")

    if 'n_tokens' in row:
        print(f"Tokens: {row['n_tokens']}")

    print(f"----------------------------------------------")

    full_text = row['lyrics']
    print(full_text)

print(f"\n==============================================")
print(f"End. Displayed {len(short_lyrics)} lyrics.")

In [ ]:
print("Analysis of 'Strange' Long Lyrics (Outliers)")

# Get the 99th percentile to define "very long"
q99 = df['n_tokens'].quantile(0.99)
print(f"The 99th percentile for n_tokens is: {q99:.0f} tokens.")

# Inspect songs above this threshold
long_lyrics = df[df['n_tokens'] > q99]

print(f"\nFound {long_lyrics.shape[0]} songs with more than {q99:.0f} tokens.")
if not long_lyrics.empty:
    print("These are the statistical outliers (long lyrics):")
    display(long_lyrics[['id', 'full_title', 'n_tokens', 'lexical_density', 'swear_IT']].sort_values('n_tokens', ascending=False))

In [ ]:
for index, row in long_lyrics.iterrows():

    print(f"\n==============================================")
    print(f"INDEX (original): {index}")

    # Print the title for context, if present
    if 'full_title' in row:
        print(f"TITLE: {row['full_title']}")

    if 'n_tokens' in row:
        print(f"Tokens: {row['n_tokens']}")

    print(f"----------------------------------------------")

    full_text = row['lyrics']
    print(full_text)

print(f"\n==============================================")
print(f"End. Displayed {len(long_lyrics)} lyrics.")

In [ ]:
print("Analysis of 'Strange' Content (Placeholders) ")

# Define common placeholder patterns
placeholder_patterns = r"Contributors|Contributor|Lyrics|COMING SOON|instrumental"

# Search for lyrics containing these patterns
placeholder_lyrics = df[df['lyrics'].str.contains(placeholder_patterns, case=False, na=False, regex=True)]

print(f"\nFound {placeholder_lyrics.shape[0]} songs containing placeholder text.")

patterns_to_check = {
    "Contributor(s)": r"Contributor"
}

print("\nIndividual placeholder counts (a song can be in multiple categories):")
for label, pattern in patterns_to_check.items():
    # Count how many lyrics contain the specific pattern
    keyword_count = df[df['lyrics'].str.contains(pattern, case=False, na=False, regex=True)].shape[0]
    print(f"  - Songs containing '{label}': {keyword_count}")

if not placeholder_lyrics.empty:
    print("Inspecting these 'strange' placeholder lyrics:")
    display(placeholder_lyrics[['id', 'full_title', 'lyrics', 'n_tokens']].head(20))

In [ ]:
for index, row in placeholder_lyrics.iterrows():

    print(f"\n==============================================")
    print(f"INDEX (original): {index}")

    # Print the title for context, if present
    if 'full_title' in row:
        print(f"TITLE: {row['full_title']}")

    if 'n_tokens' in row:
        print(f"Tokens: {row['n_tokens']}")

    print(f"----------------------------------------------")

    full_text = row['lyrics']
    print(full_text)

print(f"\n==============================================")
print(f"End. Displayed {len(placeholder_lyrics)} lyrics.")

##### Intro

In [ ]:
# Specific search for keywords
intro_pattern = 'Intro|Interlude|Outro|Prequel|Introduzione'
keywords_list = intro_pattern.split('|') # Create a list of the keywords

# Find all rows that match *any* of the keywords
intro_songs = df[df['full_title'].str.contains(intro_pattern, case=False, na=False)]

print(f"\n--- Analysis of Intro/Outro/Interlude Keywords ---")
print(f"Total rows matching at least one keyword: {len(intro_songs)}")

# --- New section for individual counts ---
print("\nIndividual keyword counts (a song can be in multiple categories):")
for keyword in keywords_list:
    # Count how many titles contain the specific keyword
    keyword_count = df[df['full_title'].str.contains(keyword, case=False, na=False)].shape[0]
    print(f"  - Titles containing '{keyword}': {keyword_count}")
# --- End of new section ---

# Display the first 20 matches, as before
if not intro_songs.empty:
    print("\nDisplaying first 20 matches:")
    display(
        intro_songs[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'duration_ms', 'album_type']].head(20))

In [ ]:
# Sort the very short songs by n_tokens (from lowest to highest)
intro_lyrics = intro_songs.sort_values(by='n_tokens', ascending=True)

print("Inspecting the intro songs")

# Print the full text of the first 5 (Note: .iterrows() will iterate all, not just 5)
for index, row in intro_lyrics.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Duration (ms): {row['duration_ms']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

#### Number of Sentence and Number of Tokens

##### Number of tokens

In [ ]:
word_threshold = 7.50
print(f"Using threshold from your IQR analysis: n_tokens < {word_threshold}")

# The pattern of 'junk' text to exclude
junk_pattern = 'Contributors|Contributor'

short_lyrics_mask = df['n_tokens'] < word_threshold

not_junk_mask = ~df['lyrics'].str.contains(junk_pattern, case=False, na=False, regex=True)

final_mask = short_lyrics_mask & not_junk_mask

short_lyrics_clean = df[final_mask]

print(f"\nTotal rows with 'short lyrics' (< {word_threshold} tokens) AND are NOT placeholders:")
print(f"Total number: {len(short_lyrics_clean)}")

if not short_lyrics_clean.empty:
    print("\nInspecting these 'clean' short lyrics (skits, interludes, or real short songs):")
    # We display the same columns you requested
    display(short_lyrics_clean[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'duration_ms']].head(20))

In [ ]:
print("--- Inspecting 'Long Lyrics' Outliers (High-End) ---")

# We use the 979.50 threshold from your IQR analysis for 'n_tokens'
word_threshold_upper = 979.50
print(f"Using upper threshold from your IQR analysis: n_tokens > {word_threshold_upper}")

long_lyrics_mask = df['n_tokens'] > word_threshold_upper

long_lyrics_df = df[long_lyrics_mask]

print(f"\nTotal rows with 'long lyrics' (> {word_threshold_upper} tokens):")
print(f"Total number found: {len(long_lyrics_df)}")

if len(long_lyrics_df) == 157:
    print("SUCCESS: The count matches your IQR analysis (157 outliers).")
else:
    print(f"NOTE: The count ({len(long_lyrics_df)}) does not match your IQR analysis (157).")
    print("This is OK, it just means the 'df' has been cleaned since the IQR was calculated.")

if not long_lyrics_df.empty:
    print("\nInspecting the 20 longest outliers (sorted by n_tokens):")

    display(
        long_lyrics_df[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'duration_ms']]
        .sort_values('n_tokens', ascending=False)
        .head(20)
    )
else:
    print("\nNo 'long lyrics' outliers found.")

In [ ]:
# Sort the very short songs by n_tokens (from lowest to highest)
long_songs = long_lyrics_df.sort_values(by='n_tokens', ascending=False)

print("Inspecting the long songs")

# Print the full text of the first 5 (Note: .iterrows() will iterate all, not just 5)
for index, row in long_songs.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Duration (ms): {row['duration_ms']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

##### Number of Sentences

In [ ]:
word_threshold = 5.50
print(f"Using threshold from your IQR analysis: n_tokens < {word_threshold}")

# The pattern of 'junk' text to exclude
junk_pattern = 'Contributors|Contributor'

short_lyrics_mask = df['n_sentences'] < word_threshold

not_junk_mask = ~df['lyrics'].str.contains(junk_pattern, case=False, na=False, regex=True)

final_mask = short_lyrics_mask & not_junk_mask

short_lyrics_clean = df[final_mask]

print(f"\nTotal rows with 'short lyrics' (< {word_threshold} sentences) AND are NOT placeholders:")
print(f"Total number: {len(short_lyrics_clean)}")

if not short_lyrics_clean.empty:
    print("\nInspecting these 'clean' short lyrics:")

    display(short_lyrics_clean[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'n_sentences']].head(20))

In [ ]:
short_songs = short_lyrics_clean.sort_values(by=['n_sentences', 'n_tokens'], ascending=True)

print("Inspecting the long songs")

for index, row in short_songs.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Sentences: {row['n_sentences']}")
    print(f"Duration (ms): {row['duration_ms']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

In [ ]:
print("--- Inspecting 'Long Lyrics' Outliers (High-End) ---")

word_threshold_upper = 113.50
print(f"Using upper threshold from your IQR analysis: n_sentences > {word_threshold_upper}")

long_lyrics_mask = df['n_sentences'] > word_threshold_upper

long_lyrics_df = df[long_lyrics_mask]

print(f"\nTotal rows with 'long lyrics' (> {word_threshold_upper} sentences):")
print(f"Total number found: {len(long_lyrics_df)}")

if len(long_lyrics_df) == 205:
    print("SUCCESS: The count matches your IQR analysis.")
else:
    print(f"NOTE: The count ({len(long_lyrics_df)}) does not match your IQR analysis.")
    print("This is OK, it just means the 'df' has been cleaned since the IQR was calculated.")

if not long_lyrics_df.empty:
    print("\nInspecting the 20 longest outliers :")

    display(
        long_lyrics_df[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'duration_ms', 'n_sentences']]
        .sort_values('n_sentences', ascending=False)
        .head(20)
    )
else:
    print("\nNo 'long lyrics' outliers found.")

In [ ]:
long_songs = long_lyrics_df.sort_values(by='n_sentences', ascending=False)

print("Inspecting the long songs")

for index, row in long_songs.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Sentences: {row['n_sentences']}")
    print(f"Duration (ms): {row['duration_ms']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

##### Coerence Check

In [ ]:
# Auto counts
n_sent_auto = []
n_tok_auto = []

for idx, lyrics in enumerate(df["lyrics"]):
    ns = functions.count_sentences(lyrics)
    nt = functions.count_tokens(lyrics)

    # Debug every 500 rows
    if idx % 500 == 0:
        print(f"[{idx}] auto_sentences={ns} auto_tokens={nt}")

    n_sent_auto.append(ns)
    n_tok_auto.append(nt)

df["n_sentences_auto"] = n_sent_auto
df["n_tokens_auto"] = n_tok_auto

# Match flags (only where ground truth is present)
if "n_sentences" in df.columns:
    df["n_sentences_match"] = df.apply(
        lambda r: (
            safe_int(r["n_sentences"]) == r["n_sentences_auto"]
            if safe_int(r["n_sentences"]) is not None
            else None
        ),
        axis=1
    )
else:
    print("WARNING: column 'n_sentences' not found in dataframe.")

if "n_tokens" in df.columns:
    df["n_tokens_match"] = df.apply(
        lambda r: (
            safe_int(r["n_tokens"]) == r["n_tokens_auto"]
            if safe_int(r["n_tokens"]) is not None
            else None
        ),
        axis=1
    )
else:
    print("WARNING: column 'n_tokens' not found in dataframe.")


In [ ]:
if "n_sentences" in df.columns:
    sent_valid_mask = df["n_sentences"].apply(lambda x: functions.safe_int(x) is not None)
    print("\n=== SENTENCE COUNT CONSISTENCY ===")
    print(f"Rows with declared n_sentences: {sent_valid_mask.sum()}")
    print(f"Rows with correct n_sentences: {(df.loc[sent_valid_mask, 'n_sentences_match'] == True).sum()}")
    print(f"Rows with WRONG n_sentences: {(df.loc[sent_valid_mask, 'n_sentences_match'] == False).sum()}")

if "n_tokens" in df.columns:
    tok_valid_mask = df["n_tokens"].apply(lambda x: functions.safe_int(x) is not None)
    print("\n=== TOKEN COUNT CONSISTENCY ===")
    print(f"Rows with declared n_tokens: {tok_valid_mask.sum()}")
    print(f"Rows with correct n_tokens: {(df.loc[tok_valid_mask, 'n_tokens_match'] == True).sum()}")
    print(f"Rows with WRONG n_tokens: {(df.loc[tok_valid_mask, 'n_tokens_match'] == False).sum()}")


display(df[['n_sentences', 'n_tokens', 'n_sentences_auto', 'n_tokens_auto']].head(50))

In [ ]:
print("We will use 'n_sentences_auto' as it is our validated column.")

# Analysis of 'Strange' Low Sentence Counts
print("Analysis of 'Strange' Low Sentence Counts")

broken_mask = df['n_tokens'].isna()
low_sentences_df = df[(df['n_sentences_auto'] <= 1) & ~broken_mask]

print(f"\nFound {low_sentences_df.shape[0]} songs with 0 or 1 sentence (excluding 'broken' rows).")
print("Hypothesis: These are the 'junk' placeholder lyrics (e.g., '1 Contributor Lyrics').")

if not low_sentences_df.empty:
    print("Inspecting a sample of these 'low sentence' lyrics:")
    display(low_sentences_df[['id', 'full_title', 'lyrics', 'n_sentences_auto', 'n_tokens_auto']].head(10))
else:
    print("No 'low sentence' outliers found (excluding broken rows).")


In [ ]:
import warnings

print("Cross-Analysis: Low Sentences vs. Placeholders")

warnings.filterwarnings('ignore', category=FutureWarning)

# Define Group A: Placeholders
placeholder_patterns = r"Contributors|Contributor|Lyrics|COMING SOON|instrumental"
placeholder_mask = df['lyrics'].str.contains(placeholder_patterns, case=False, na=False, regex=True)
total_placeholders = placeholder_mask.sum()

# Define Group B: Low Sentences
broken_mask = df['n_tokens'].isna()
low_sent_mask = (df['n_sentences_auto'] <= 1) & ~broken_mask
total_low_sentences = low_sent_mask.sum()

# Calculate Intersection (Venn Diagram)
overlap_mask = placeholder_mask & low_sent_mask
overlap_count = overlap_mask.sum()

low_sent_only_mask = low_sent_mask & ~placeholder_mask
low_sent_only_count = low_sent_only_mask.sum()

placeholder_only_mask = placeholder_mask & ~low_sent_mask
placeholder_only_count = placeholder_only_mask.sum()

# Print Text Results
print(f"Total songs with placeholders (Group A): {total_placeholders}")
print(f"Total songs with 0 or 1 sentence (Group B): {total_low_sentences}")
print("\n--- Venn Diagram Results ---")
print(f"Songs IN BOTH Group A AND Group B (Overlap - 'Junk'): {overlap_count}")
print(f"Songs with low sentences BUT NO placeholder (Group B only - 'Strange'): {low_sent_only_count}")
print(f"Songs with placeholders BUT > 1 sentence (Group A only - 'Contaminated'): {placeholder_only_count}")

#  Visualization
print("Visualizing the Overlap")

plot_data = pd.DataFrame([
    {'category': f'Placeholders Only (Contaminated)',
     'description': '"Contaminated" lyrics (> 1 sentence)',
     'count': placeholder_only_count},
    {'category': f'Low Sentences Only (Strange)',
     'description': '"Strange" lyrics but not placeholders (<= 1 sentence)',
     'count': low_sent_only_count},
    {'category': f'Overlap (Junk)',
     'description': '"Junk" lyrics (placeholder AND <= 1 sentence)',
     'count': overlap_count}
])

# Create the Altair bar chart
chart = alt.Chart(plot_data).mark_bar().encode(
    x=alt.X('category:N', title='Category (Venn Analysis)', sort='-y'), # Sort by count
    y=alt.Y('count:Q', title='Number of Songs'),
    color=alt.Color('category:N', title='Category'),
    tooltip=['category', 'description', 'count']
).properties(
    title='Cross-Analysis: Placeholders vs. Short Sentences'
)

# Add numeric labels above the bars
text = chart.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Move text 5 pixels above the bar
).encode(
    text='count:Q'
)

final_chart = (chart + text).interactive()


final_chart

This cross-analysis confirms that our two "low-quality" data groups are highly related but distinct.

The 114-song overlap (Overlap (Junk)) confirms that the vast majority of tracks with n_sentences_auto <= 1 are "junk" data (e.g., "1 Contributor Lyrics"). These will be removed.

The 41 Placeholders Only songs are "contaminated" tracks. As noted, these contain valid lyrics mixed with placeholder text and will be cleaned (not removed).

The 7 Low Sentences Only songs are "strange" but valid outliers (e.g., short interludes) that will also be removed as they lack sufficient content for analysis.

#### Number Token per  Sentence

In [ ]:
word_threshold = 4.19
print(f"Using threshold from your IQR analysis: tokens_per_sent < {word_threshold}")

# The pattern of 'junk' text to exclude
junk_pattern = 'Contributors|Contributor'

short_lyrics_mask = df['tokens_per_sent'] < word_threshold

not_junk_mask = ~df['lyrics'].str.contains(junk_pattern, case=False, na=False, regex=True)

final_mask = short_lyrics_mask & not_junk_mask

short_lyrics_clean = df[final_mask]

print(f"\nTotal rows with 'short lyrics' (< {word_threshold} token per sentence) AND are NOT placeholders:")
print(f"Total number: {len(short_lyrics_clean)}")

if not short_lyrics_clean.empty:
    print("\nInspecting these 'clean' short lyrics:")
    # We display the same columns you requested
    display(short_lyrics_clean[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'n_sentences', 'tokens_per_sent']].head(20))

In [ ]:
short_songs = short_lyrics_clean.sort_values(by='tokens_per_sent', ascending=True)

print("Inspecting the long songs")

for index, row in short_songs.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Sentences: {row['n_sentences']}")
    print(f"Token per sentence: {row['tokens_per_sent']}")
    print(f"Duration (ms): {row['duration_ms']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

In [ ]:
print("--- Inspecting 'Long Lyrics' Outliers (High-End) ---")

word_threshold_upper = 12.58
print(f"Using upper threshold from your IQR analysis: tokens_per_sent > {word_threshold_upper}")

long_lyrics_mask = df['tokens_per_sent'] > word_threshold_upper

long_lyrics_df = df[long_lyrics_mask]

print(f"\nTotal rows with 'long lyrics' (> {word_threshold_upper} token per sentence):")
print(f"Total number found: {len(long_lyrics_df)}")

if len(long_lyrics_df) == 283:
    print("SUCCESS: The count matches your IQR analysis.")
else:
    print(f"NOTE: The count ({len(long_lyrics_df)}) does not match your IQR analysis.")
    print("This is OK, it just means the 'df' has been cleaned since the IQR was calculated.")

if not long_lyrics_df.empty:
    print("\nInspecting the 20 longest outliers :")

    display(
        long_lyrics_df[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'duration_ms', 'n_sentences', 'tokens_per_sent']]
        .sort_values('tokens_per_sent', ascending=False)
        .head(20)
    )
else:
    print("\nNo 'long lyrics' outliers found.")

In [ ]:
long_songs = long_lyrics_df.sort_values(by='tokens_per_sent', ascending=False)

print("Inspecting the long songs")

for index, row in long_songs.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Sentences: {row['n_sentences']}")
    print(f"Token per sent: {row['tokens_per_sent']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

##### Coerence Check

In [ ]:
print("Recalculating 'tokens_per_sent_auto' using n_tokens_auto / n_sentences_auto...")

df['tokens_per_sent_auto'] = np.where(
    df['n_sentences_auto'] == 0,
    0,  # Imposta a 0 se n_sentences_auto è 0
    df['n_tokens_auto'] / df['n_sentences_auto']
)

print("Comparing new 'tokens_per_sent_auto' with original 'tokens_per_sent'...")

if "tokens_per_sent" in df.columns:

    valid_mask = df['tokens_per_sent'].notna()

    match_series = np.isclose(
        df.loc[valid_mask, 'tokens_per_sent'],
        df.loc[valid_mask, 'tokens_per_sent_auto'],
        atol=0.01
    )

    df['tokens_per_sent_match'] = pd.NA
    df.loc[valid_mask, 'tokens_per_sent_match'] = match_series

    print("\n=== tokens_per_sent CONSISTENCY ===")
    print(f"Rows with declared tokens_per_sent: {valid_mask.sum()}")
    print(f"Rows with correct tokens_per_sent (within 0.01 tolerance): {(df['tokens_per_sent_match'] == True).sum()}")
    print(f"Rows with WRONG tokens_per_sent: {(df['tokens_per_sent_match'] == False).sum()}")

else:
    print("WARNING: column 'tokens_per_sent' not found in dataframe.")

print("We compare the original 'tokens_per_sent' with our 'tokens_per_sent_auto'.")
display(df[[
    'tokens_per_sent', 'tokens_per_sent_auto', 'tokens_per_sent_match',
    'n_sentences', 'n_sentences_auto',
    'n_tokens', 'n_tokens_auto'
]].head(20))


print("Verifying that 'tokens_per_sent_auto' for 'junk' data.")
#
display(df[df['lyrics'].str.contains(r"Contributors|COMING SOON", na=False)][[
    'lyrics',
    'n_sentences_auto',
    'n_tokens_auto',
    'tokens_per_sent',
    'tokens_per_sent_auto'
]].head(20))

#### Number of char per token


In [ ]:
word_threshold = 3.41
print(f"Using threshold from your IQR analysis: char_per_tok < {word_threshold}")

# The pattern of 'junk' text to exclude
junk_pattern = 'Contributors|Contributor'

short_lyrics_mask = df['char_per_tok'] < word_threshold

not_junk_mask = ~df['lyrics'].str.contains(junk_pattern, case=False, na=False, regex=True)

final_mask = short_lyrics_mask & not_junk_mask

short_lyrics_clean = df[final_mask]

print(f"\nTotal rows with 'short lyrics' (< {word_threshold} char per token) AND are NOT placeholders:")
print(f"Total number: {len(short_lyrics_clean)}")

if not short_lyrics_clean.empty:
    print("\nInspecting these 'clean' short lyrics:")
    # We display the same columns you requested
    display(short_lyrics_clean[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'n_sentences', 'tokens_per_sent', 'char_per_tok']].head(20))

In [ ]:
short_songs = short_lyrics_clean.sort_values(by='char_per_tok', ascending=True)

print("Inspecting the long songs")

for index, row in short_songs.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Sentences: {row['n_sentences']}")
    print(f"Char per sent: {row['char_per_tok']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

In [ ]:
print("--- Inspecting 'Long Lyrics' Outliers (High-End) ---")

word_threshold_upper = 4.62
print(f"Using upper threshold from your IQR analysis: char_per_tok > {word_threshold_upper}")

junk_pattern = 'Contributors|Contributor'

long_lyrics_mask = df['char_per_tok'] > word_threshold_upper


not_junk_mask = ~df['lyrics'].str.contains(junk_pattern, case=False, na=False, regex=True)

final_mask = long_lyrics_mask & not_junk_mask

long_lyrics_df = df[final_mask]

print(f"\nTotal rows with 'long lyrics' (> {word_threshold_upper} char per token):")
print(f"Total number found: {len(long_lyrics_df)}")

if len(long_lyrics_df) == 330:
    print("SUCCESS: The count matches your IQR analysis.")
else:
    print(f"NOTE: The count ({len(long_lyrics_df)}) does not match your IQR analysis.")
    print("This is OK, it just means the 'df' has been cleaned since the IQR was calculated.")

if not long_lyrics_df.empty:
    print("\nInspecting the 20 longest outliers :")

    display(
        long_lyrics_df[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'duration_ms', 'n_sentences', 'tokens_per_sent', 'char_per_tok' ]]
        .sort_values('char_per_tok', ascending=False)
        .head(20)
    )
else:
    print("\nNo 'long lyrics' outliers found.")

In [ ]:
long_songs = long_lyrics_df.sort_values(by='char_per_tok', ascending=False)

print("Inspecting the long songs")

for index, row in long_songs.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Sentences: {row['n_sentences']}")
    print(f"Token per sent: {row['tokens_per_sent']}")
    print(f"Char per sent: {row['char_per_tok']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

##### Coerence Check

In [ ]:
# Recalculate 'char_per_tok'
print("Calculating 'total_chars_auto' (sum of all token lengths)...")
df['total_chars_auto'] = df['lyrics'].apply(functions.count_total_token_chars)

print("Recalculating 'char_per_tok_auto' using total_chars_auto / n_tokens_auto...")

df['char_per_tok_auto'] = np.where(
    df['n_tokens_auto'] == 0,
    0,
    df['total_chars_auto'] / df['n_tokens_auto']
)

# Compare 'char_per_tok_auto' with the original 'char_per_tok'
print("Comparing new 'char_per_tok_auto' with original 'char_per_tok'...")

if "char_per_tok" in df.columns:
    valid_mask = df['char_per_tok'].notna()

    match_series = np.isclose(
        df.loc[valid_mask, 'char_per_tok'],
        df.loc[valid_mask, 'char_per_tok_auto'],
        atol=0.01
    )

    df['char_per_tok_match'] = pd.NA
    df.loc[valid_mask, 'char_per_tok_match'] = match_series

    print("\n=== char_per_tok CONSISTENCY ===")
    print(f"Rows with declared char_per_tok: {valid_mask.sum()}")
    print(f"Rows with correct char_per_tok (within 0.01 tolerance): {(df['char_per_tok_match'] == True).sum()}")
    print(f"Rows with WRONG char_per_tok: {(df['char_per_tok_match'] == False).sum()}")

else:
    print("WARNING: column 'char_per_tok' not found in dataframe.")

print("We compare the original 'char_per_tok' with our 'char_per_tok_auto'.")
display(df[[
    'char_per_tok', 'char_per_tok_auto', 'char_per_tok_match',
    'n_tokens', 'n_tokens_auto',
    'total_chars_auto'
]].head(20))

print("Verifying 'char_per_tok_auto' for 'junk' data.")
display(df[df['lyrics'].str.contains(r"Contributors|COMING SOON", na=False)][[
    'lyrics',
    'n_tokens_auto',
    'total_chars_auto',
    'char_per_tok',
    'char_per_tok_auto'
]].head(5))

#### Lexical density

In [ ]:
word_threshold = 0.39
print(f"Using threshold from your IQR analysis: lexical_density < {word_threshold}")

# The pattern of 'junk' text to exclude
junk_pattern = 'Contributors|Contributor'

short_lyrics_mask = df['lexical_density'] < word_threshold

not_junk_mask = ~df['lyrics'].str.contains(junk_pattern, case=False, na=False, regex=True)

final_mask = short_lyrics_mask & not_junk_mask

short_lyrics_clean = df[final_mask]

print(f"\nTotal rows with 'short lyrics' (< {word_threshold} lexical density) AND are NOT placeholders:")
print(f"Total number: {len(short_lyrics_clean)}")

if not short_lyrics_clean.empty:
    print("\nInspecting these 'clean' short lyrics:")
    # We display the same columns you requested
    display(short_lyrics_clean[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'n_sentences', 'tokens_per_sent', 'char_per_tok', 'lexical_density']].head(20))

In [ ]:
short_songs = short_lyrics_clean.sort_values(by='lexical_density', ascending=True)

print("Inspecting the long songs")

for index, row in short_songs.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Sentences: {row['n_sentences']}")
    print(f"Char per sent: {row['char_per_tok']}")
    print(f"Lexical density: {row['lexical_density']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

In [ ]:
print("--- Inspecting 'Long Lyrics' Outliers (High-End) ---")

word_threshold_upper = 0.63
print(f"Using upper threshold from your IQR analysis: char_per_tok > {word_threshold_upper}")

long_lyrics_mask = df['char_per_tok'] > word_threshold_upper

long_lyrics_df = df[long_lyrics_mask]

print(f"\nTotal rows with 'long lyrics' (> {word_threshold_upper} char per token):")
print(f"Total number found: {len(long_lyrics_df)}")

if len(long_lyrics_df) == 318:
    print("SUCCESS: The count matches your IQR analysis.")
else:
    print(f"NOTE: The count ({len(long_lyrics_df)}) does not match your IQR analysis.")
    print("This is OK, it just means the 'df' has been cleaned since the IQR was calculated.")

if not long_lyrics_df.empty:
    print("\nInspecting the 20 longest outliers :")

    display(
        long_lyrics_df[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'duration_ms', 'n_sentences', 'tokens_per_sent', 'char_per_tok', 'lexical_density' ]]
        .sort_values('lexical_density', ascending=False)
        .head(20)
    )
else:
    print("\nNo 'long lyrics' outliers found.")

In [ ]:
long_songs = long_lyrics_df.sort_values(by='lexical_density', ascending=False)

print("Inspecting the long songs")

for index, row in long_songs.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Sentences: {row['n_sentences']}")
    print(f"Token per sent: {row['tokens_per_sent']}")
    print(f"Char per sent: {row['char_per_tok']}")
    print(f"Lexical density: {row['lexical_density']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

##### Coerence Check

In [ ]:
print("Calculating 'n_unique_words_auto'...")
df['n_unique_words_auto'] = df['lyrics'].apply(functions.count_unique_tokens)

print("Recalculating 'lexical_density_auto' using n_unique_words_auto / n_tokens_auto...")

df['lexical_density_auto'] = np.where(
    df['n_tokens_auto'] == 0,
    0,
    df['n_unique_words_auto'] / df['n_tokens_auto']
)

print("Comparing new 'lexical_density_auto' with original 'lexical_density'...")

if "lexical_density" in df.columns:
    valid_mask = df['lexical_density'].notna()

    match_series = np.isclose(
        df.loc[valid_mask, 'lexical_density'],
        df.loc[valid_mask, 'lexical_density_auto'],
        atol=0.01
    )

    df['lexical_density_match'] = pd.NA
    df.loc[valid_mask, 'lexical_density_match'] = match_series

    print("\n=== lexical_density CONSISTENCY ===")
    print(f"Rows with declared lexical_density: {valid_mask.sum()}")
    print(f"Rows with correct lexical_density (within 0.01 tolerance): {(df['lexical_density_match'] == True).sum()}")
    print(f"Rows with WRONG lexical_density: {(df['lexical_density_match'] == False).sum()}")

else:
    print("WARNING: column 'lexical_density' not found in dataframe.")


display(df[[
    'lexical_density', 'lexical_density_auto', 'lexical_density_match',
    'n_tokens', 'n_tokens_auto',
    'n_unique_words_auto'
]].head(10))

print("Verifying 'lexical_density_auto' for 'junk' data.")
display(df[df['lyrics'].str.contains(r"Contributors|COMING SOON", na=False)][[
    'lyrics',
    'n_tokens_auto',
    'n_unique_words_auto',
    'lexical_density_auto'
]].head(5))

#### Average token per clause

In [ ]:
word_threshold = 2.51
print(f"Using threshold from your IQR analysis: avg_token_per_clause < {word_threshold}")

# The pattern of 'junk' text to exclude
junk_pattern = 'Contributors|Contributor'

short_lyrics_mask = df['avg_token_per_clause'] < word_threshold

not_junk_mask = ~df['lyrics'].str.contains(junk_pattern, case=False, na=False, regex=True)

final_mask = short_lyrics_mask & not_junk_mask

short_lyrics_clean = df[final_mask]

print(f"\nTotal rows with 'short lyrics' (< {word_threshold} average token per clause) AND are NOT placeholders:")
print(f"Total number: {len(short_lyrics_clean)}")

if not short_lyrics_clean.empty:
    print("\nInspecting these 'clean' short lyrics:")
    # We display the same columns you requested
    display(short_lyrics_clean[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'n_sentences', 'tokens_per_sent', 'char_per_tok', 'lexical_density', 'avg_token_per_clause']].head(20))

In [ ]:
short_songs = short_lyrics_clean.sort_values(by='avg_token_per_clause', ascending=True)

print("Inspecting the short songs")

for index, row in short_songs.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Sentences: {row['n_sentences']}")
    print(f"Char per sent: {row['char_per_tok']}")
    print(f"Lexical density: {row['lexical_density']}")
    print(f"Token per clause: {row['avg_token_per_clause']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

In [ ]:
print("--- Inspecting 'Long Lyrics' Outliers (High-End) ---")

word_threshold_upper = 11.44
print(f"Using upper threshold from your IQR analysis: avg_token_per_clause > {word_threshold_upper}")

long_lyrics_mask = df['avg_token_per_clause'] > word_threshold_upper

long_lyrics_df = df[long_lyrics_mask]

print(f"\nTotal rows with 'long lyrics' (> {word_threshold_upper} average token per clause):")
print(f"Total number found: {len(long_lyrics_df)}")

if len(long_lyrics_df) == 605:
    print("SUCCESS: The count matches your IQR analysis.")
else:
    print(f"NOTE: The count ({len(long_lyrics_df)}) does not match your IQR analysis.")
    print("This is OK, it just means the 'df' has been cleaned since the IQR was calculated.")

if not long_lyrics_df.empty:
    print("\nInspecting the 20 longest outliers :")

    display(
        long_lyrics_df[['id', 'name_artist', 'full_title', 'lyrics', 'n_tokens', 'duration_ms', 'n_sentences', 'tokens_per_sent', 'char_per_tok', 'lexical_density', 'avg_token_per_clause' ]]
        .sort_values('avg_token_per_clause', ascending=False)
        .head(20)
    )
else:
    print("\nNo 'long lyrics' outliers found.")

In [ ]:
long_songs = long_lyrics_df.sort_values(by='avg_token_per_clause', ascending=False)

print("Inspecting the long songs")

for index, row in long_songs.iterrows():
    print("\n==============================================")
    print(f"INDEX (original): {index}")
    print(f"ARTIST: {row['name_artist']}")
    print(f"TITLE: {row['full_title']}")
    print(f"Tokens: {row['n_tokens']}")
    print(f"Sentences: {row['n_sentences']}")
    print(f"Token per sent: {row['tokens_per_sent']}")
    print(f"Char per sent: {row['char_per_tok']}")
    print(f"Lexical density: {row['lexical_density']}")
    print(f"Token per clause: {row['avg_token_per_clause']}")
    print("----------------------------------------------")
    print(f"LYRICS:\n'{str(row['lyrics'])}'")

## DATA CORRELATION BEFORE CLEANING

 In this section we:
 - select numeric columns
 - compute Pearson and Spearman correlation matrices
 - detect highly correlated pairs (potentially redundant features)

In [ ]:
# Select only numeric columns (optionally exclude IDs)
id_like = {"id", "id_track", "id_album", "id_album_clean", "id_artist"}
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in id_like]

print("Numeric columns used:", num_cols)

### Pearson correlation (linear relationships)

In [ ]:
# =========================
# Column filters for Pearson
# =========================
id_like = {"id", "id_track", "id_album", "id_album_clean", "id_artist"}

# =========================
# 1) Select numeric candidates (exclude id-like)
# =========================
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in id_like]

excluded = {}
kept = []

for c in num_cols:
    s = df[c]
    reasons = []
    if pd.api.types.is_bool_dtype(s) or functions.is_binary_series(s):
        reasons.append("binary/bool → use point-biserial (not raw Pearson)")
    if functions.is_low_cardinal_int(s, max_levels=5):
        reasons.append("low-cardinality ordinal → prefer Spearman/Kendall")
    if functions.is_quasi_constant(s):
        reasons.append("quasi-constant → variance ~ 0")
    if functions.is_extreme_zeroinflated_count(s):
        reasons.append("extreme zero-inflated count")
    if reasons:
        excluded[c] = "; ".join(reasons)
    else:
        kept.append(c)

print("Columns kept for Pearson:", kept)
if excluded:
    print("\nColumns excluded and reason:")
    for k, v in excluded.items():
        print(f"- {k}: {v}")

# =========================
# 2) Pearson correlation (only 'kept' columns)
# =========================
if len(kept) < 2:
    raise ValueError("Too many columns excluded: fewer than 2 columns suitable for Pearson.")

pearson_corr = df[kept].corr(method="pearson")
print("\nPearson correlation (filtered):")
print(pearson_corr)

# =========================
# 3) Strong correlations (pair list, NOT matrix)
# =========================
threshold = 0.9  # adjust as needed

upper_mask = np.triu(np.ones(pearson_corr.shape), k=1).astype(bool)
pairs = (
    pearson_corr.where(upper_mask)
    .stack()
    .reset_index()
)
pairs.columns = ["feature_1", "feature_2", "pearson_r"]

strong_pearson = (
    pairs[pairs["pearson_r"].abs() >= threshold]
    .sort_values(by="pearson_r", key=lambda s: s.abs(), ascending=False)
)
print(f"\nStrong Pearson pairs (|r| >= {threshold}):")
print(strong_pearson if not strong_pearson.empty else "None")


In [ ]:
# ===== Custom palette =====

hex_colors = ["#e1bee7", "#ce93d8", "#9c27b0"]  # your colors
cmap_custom = LinearSegmentedColormap.from_list("custom_purple", hex_colors)

# =========================
# Heatmap (Pearson matrix) with custom colors
# =========================
# Uses the same column order as `kept`
corr_mat = pearson_corr.loc[kept, kept]

# Mask NaNs to show as empty cells
data = np.ma.masked_invalid(corr_mat.to_numpy())

plt.figure(figsize=(8, 8))
# TwoSlopeNorm centers the color at 0, with vmin=-1, vmax=1
norm = TwoSlopeNorm(vmin=-1, vcenter=0, vmax=1)
im = plt.imshow(data, aspect="auto", norm=norm, cmap=cmap_custom)
plt.colorbar(im, label="Pearson r")

plt.xticks(ticks=np.arange(len(kept)), labels=kept, rotation=90)
plt.yticks(ticks=np.arange(len(kept)), labels=kept)

# Optional grid
ax = plt.gca()
ax.set_xticks(np.arange(-.5, len(kept), 1), minor=True)
ax.set_yticks(np.arange(-.5, len(kept), 1), minor=True)
ax.grid(which="minor", linestyle="-", linewidth=0.5, alpha=0.25)
ax.tick_params(which="minor", bottom=False, left=False)

plt.title("Pearson correlation matrix (filtered)")
plt.tight_layout()
plt.show()


#### Spearman correlation (monotonic relationships, robust to outliers)

In [ ]:
# =========================
# Column filters for Spearman + Pearson vs Spearman comparison
# =========================

id_like = {"id", "id_track", "id_album", "id_album_clean", "id_artist"}

# Variables typically circular (Spearman is not recommended for cyclic relationships)
circular_names = {"month", "day", "weekday", "dow", "hour", "minute", "second"}

# =========================
# 1) Select numeric candidates for Spearman (keep ordinal/binary, exclude id-like)
# =========================
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in id_like]

excluded_s = {}
kept_s = []
spearman_soft_warn = {}

for c in num_cols:
    s = df[c]
    reasons = []
    # Hard exclusions for Spearman suitability
    if functions.is_quasi_constant(s) or s.dropna().nunique() < 2:
        reasons.append("constant/nearly-constant or <2 distinct non-null values")
    if c.lower() in circular_names:
        reasons.append("circular variable (cyclic) — Spearman not suitable")
    if reasons:
        excluded_s[c] = "; ".join(reasons)
        continue

    # Keep the column
    kept_s.append(c)

    # Soft warnings (still kept, but flagged)
    warns = []
    if pd.api.types.is_integer_dtype(s) and (s >= 0).all() and (s == 0).mean() >= 0.95:
        warns.append("extreme zero-inflation (many ties) — Kendall τ-b or transformations may be better")
    if warns:
        spearman_soft_warn[c] = "; ".join(warns)

print("Columns kept for Spearman:", kept_s)
if excluded_s:
    print("\nColumns excluded for Spearman (hard) and reason:")
    for k, v in excluded_s.items():
        print(f"- {k}: {v}")
if spearman_soft_warn:
    print("\nColumns where Spearman is NOT recommended (soft warning):")
    for k, v in spearman_soft_warn.items():
        print(f"- {k}: {v}")

# =========================
# 2) Spearman correlation (only 'kept_s' columns)
# =========================
if len(kept_s) < 2:
    raise ValueError("Too few columns for Spearman: need at least 2 usable columns.")

# Use min_periods to reduce all-NaN issues with sparse overlap
spearman_corr = df[kept_s].corr(method="spearman", min_periods=3)
print("\nSpearman correlation (filtered):")
print(spearman_corr)

# =========================
# 3) Strong correlations (pair list, NOT matrix)
# =========================
threshold = 0.9  # adjust as needed

upper_mask_s = np.triu(np.ones(spearman_corr.shape), k=1).astype(bool)
pairs_s = (
    spearman_corr.where(upper_mask_s)
    .stack()
    .reset_index()
)
pairs_s.columns = ["feature_1", "feature_2", "spearman_rho"]

strong_spearman = (
    pairs_s[pairs_s["spearman_rho"].abs() >= threshold]
    .sort_values(by="spearman_rho", key=lambda s: s.abs(), ascending=False)
)

print(f"\nStrong Spearman pairs (|rho| >= {threshold}):")
print(strong_spearman if not strong_spearman.empty else "None")

# =========================
# 4) Pearson vs Spearman comparison on common pairs
#    (requires you already computed `pearson_corr` and `strong_pearson`)
# =========================
def _pairs_from_corr(corr_df: pd.DataFrame, val_name: str) -> pd.DataFrame:
    """Stack upper triangle of a correlation matrix into a tidy pair list."""
    mask = np.triu(np.ones(corr_df.shape), k=1).astype(bool)
    out = (
        corr_df.where(mask)
        .stack()
        .reset_index()
    )
    out.columns = ["feature_1", "feature_2", val_name]
    # Ensure consistent pair ordering (feature_1 < feature_2) to merge reliably
    ordered = out.apply(
        lambda r: pd.Series(sorted([r["feature_1"], r["feature_2"]]) + [r[val_name]]),
        axis=1
    )
    ordered.columns = ["f1", "f2", val_name]
    return ordered

try:
    # Intersect the variable sets, so we're comparing apples to apples
    common_cols = sorted(set(pearson_corr.columns).intersection(set(spearman_corr.columns)))
    if len(common_cols) >= 2:
        p_pairs = _pairs_from_corr(pearson_corr.loc[common_cols, common_cols], "pearson_r")
        s_pairs = _pairs_from_corr(spearman_corr.loc[common_cols, common_cols], "spearman_rho")
        comp = p_pairs.merge(s_pairs, on=["f1", "f2"], how="inner")
        comp["abs_delta"] = (comp["pearson_r"] - comp["spearman_rho"]).abs()
        comp["sign_disagreement"] = np.sign(comp["pearson_r"]) != np.sign(comp["spearman_rho"])

        # Sort by absolute delta descending
        comp_sorted = comp.sort_values("abs_delta", ascending=False).reset_index(drop=True)

        print("\nPearson vs Spearman comparison (common pairs):")
        print(comp_sorted.head(30).to_string(index=False))

        # Optional: also show only sign disagreements
        comp_sign_flip = comp_sorted[comp_sorted["sign_disagreement"]]
        if not comp_sign_flip.empty:
            print("\nPairs with SIGN DISAGREEMENT between Pearson and Spearman:")
            print(comp_sign_flip.head(30).to_string(index=False))

        # Expose for later use (e.g., plotting)
        comparison_table = comp_sorted.copy()
    else:
        print("\nNot enough common columns for comparison.")
except NameError:
    print("\n`pearson_corr` not found. Run the Pearson block first to enable the comparison.")


In [ ]:
# Custom palette
cmap_custom = LinearSegmentedColormap.from_list("custom_purple", hex_colors)

# Ensure same column order as kept_s
corr_mat_s = spearman_corr.loc[kept_s, kept_s]

# Mask NaNs so they appear as empty cells
data_s = np.ma.masked_invalid(corr_mat_s.to_numpy())

plt.figure(figsize=(8, 8))
norm = TwoSlopeNorm(vmin=-1, vcenter=0, vmax=1)  # center color at 0
im = plt.imshow(data_s, aspect="auto", norm=norm, cmap=cmap_custom)
plt.colorbar(im, label="Spearman ρ")

plt.xticks(ticks=np.arange(len(kept_s)), labels=kept_s, rotation=90)
plt.yticks(ticks=np.arange(len(kept_s)), labels=kept_s)

# Optional grid to separate cells
ax = plt.gca()
ax.set_xticks(np.arange(-.5, len(kept_s), 1), minor=True)
ax.set_yticks(np.arange(-.5, len(kept_s), 1), minor=True)
ax.grid(which="minor", linestyle="-", linewidth=0.5, alpha=0.25)
ax.tick_params(which="minor", bottom=False, left=False)

plt.title("Spearman correlation matrix (filtered)")
plt.tight_layout()
plt.show()